In [1]:
!pip install openai transformers datasets rouge_score nltk numpy torch flash_attn

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 62.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=e793866b2cb112f570fb4c261dfce6a0748aa8031ef661e46887cce543742e8b
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
  Created wheel for flash_attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=256040057 sha256=f25da18657a87fc83dc1bfb8b7751b82246e9db355510226b674fd437c34b5fb
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built rouge_score flash_attn


In [7]:
from huggingface_hub import login
import torch
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer
from sklearn.metrics import precision_score, recall_score, f1_score
from rouge_score import rouge_scorer
import re
import json
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import numpy as np

torch.cuda.empty_cache()
# Log in with access token
# to be replaced
login(token="hf_your_access_token_here")

Access: ZHcfFFsrftNHdCIGXgzyRosnfjZOEGNlYz

- load dataset

In [8]:
dataset = load_dataset("sobamchan/aclsum", "extractive")["test"]
val_dataset = load_dataset("sobamchan/aclsum", "extractive")["validation"]
print(dataset[0].keys())

README.md: 0.00B [00:00, ?B/s]

train.jsonl: 0.00B [00:00, ?B/s]

val.jsonl: 0.00B [00:00, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/50 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

dict_keys(['id', 'source_sentences', 'challenge_sentences', 'approach_sentences', 'outcome_sentences', 'challenge_labels', 'approach_labels', 'outcome_labels'])


In [9]:
print(dataset)

Dataset({
    features: ['id', 'source_sentences', 'challenge_sentences', 'approach_sentences', 'outcome_sentences', 'challenge_labels', 'approach_labels', 'outcome_labels'],
    num_rows: 100
})


In [10]:
print(dataset['challenge_labels'][0])

[1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


- create correct gold standard

In [11]:
# Aspects to analyze
ASPECTS = ["challenge", "approach", "outcome"]

def labels_to_indices(labels):
    return [i+1 for i, v in enumerate(labels) if int(v) == 1]

def gold_for_doc(example):
    return {
        "challenge": labels_to_indices(example["challenge_labels"]),
        "approach":  labels_to_indices(example["approach_labels"]),
        "outcome":   labels_to_indices(example["outcome_labels"]),
    }

def gold_for_dataset(ds):
    return [gold_for_doc(ex) for ex in ds]

def indices_to_text(sentences, idxs_1b):
    return " ".join(sentences[i-1] for i in idxs_1b if 1 <= i <= len(sentences))

gold_all = gold_for_dataset(dataset)
print(gold_all[0])

{'challenge': [1, 2, 7, 11], 'approach': [3, 15, 19, 26, 27], 'outcome': [4, 5, 20, 21, 30, 31]}


- capping to try to improve predictions - this avoids overprediction for the model and it will drasticly improve precision. What was notices is that consistently, both models regardless of prompting technique tend to predict a number of indices which is much larger than the gold standard. Capping implies using the median gold sentences per aspect to restrict the number of predicted indices. This is what the following functions do.
- all prompting techniques have both the capped and not capped version

In [12]:
def aspect_caps_from_gold(gold_all, aspects=ASPECTS, min_cap=1):
    """
    Derive a sensible per-aspect cap K using the median #gold sentences per aspect.
    Ensures at least `min_cap` (default 1).
    Returns: dict like {"challenge": 4, "approach": 5, "outcome": 5}
    """
    counts = {a: [] for a in aspects}
    for g in gold_all:
        for a in aspects:
            counts[a].append(len(g.get(a, [])))
    return {a: max(min_cap, int(np.median(counts[a]) or 0)) for a in aspects}

# Build once after you computed `gold_all`
CAPS = aspect_caps_from_gold(gold_all)
print("Caps:", CAPS)

Caps: {'challenge': 4, 'approach': 6, 'outcome': 4}


In [13]:
def _cap_indices_in_order(indices, cap):
    """Keep at most `cap` indices, preserving ascending order."""
    if cap is None or cap <= 0:
        return indices
    return indices[:cap]

In [14]:
def make_capped_renderer(base_renderer, caps_dict):
    """
    base_renderer: a function like vanilla_prompt(sentences, target_label)
    caps_dict: e.g., CAPS from step 1
    Returns a new renderer with an extra constraint appended to the prompt.
    """
    def capped_renderer(sentences, target_label):
        cap = caps_dict.get(target_label)
        prompt = base_renderer(sentences, target_label)
        # Append a crisp constraint the model will see
        return (
            prompt
            + f"\n\nAdditional constraint:\n"
              f"- Select AT MOST {cap} sentences for the \"{target_label}\" aspect."
        )
    return capped_renderer


- prompting techniques

Vanilla prompt is the most basic version of the prompt and it is constructed as indicated by the instructions

In [15]:
# vanilla prompt
def vanilla_prompt(sentences, target_label):
    # target_label ∈ {"challenge","approach","outcome"}
    numbered = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]
    input_text = "\n".join(numbered)
    return f'''You are an expert in extractive summarization. Your task is to select sentences that express the "{target_label}" aspect of the document.

Aspect definitions:
- challenge: problem, gap, limitation, unmet need, difficulty/motivation.
- approach: method, model, system, algorithm, dataset design, procedure.
- outcome: results, findings, improvements, metrics, performance, impact.

Input:
{input_text}

Rules:
- Select ONLY sentences that primarily express the "{target_label}" aspect.
- If none match, return an empty list.
- Indices are 1-based.
- Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''


In [16]:
vanilla_capped = make_capped_renderer(vanilla_prompt, CAPS)

**least to most**: The document is broken into thematic sections (e.g., problem, method, results), and representative sentences are extracted from each.

All more complex prompting techniques are also brought in their one shot version, which takes a random instance from the validation dataset (with set random seed for reproducibility) and uses it as an example. The one-shot versions can be changed into few shot by raising the k value in order to collect more data from the validation dataset. This was not tested due to the problem becoming too computaitionally complex and receiving Out Of Memory error.

In [17]:
def least_to_most_prompt(sentences, target_label):
    numbered = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]
    input_text = "\n".join(numbered)

    return f'''You are an expert in extractive summarization. Your task is to select sentences that express the "{target_label}" aspect of the document.

Aspect definitions:
- challenge: problem, gap, limitation, unmet need, difficulty/motivation.
- approach: method, model, system, algorithm, dataset design, procedure.
- outcome: results, findings, improvements, metrics, performance, impact.

Input:
{input_text}

First, consider the overall purpose of the document and how each sentence contributes to it.
Then, from that understanding, select ONLY sentences that primarily express the "{target_label}" aspect.
If none match, return an empty list.
Indices are 1-based.
Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''

In [18]:
least_to_most_capped = make_capped_renderer(least_to_most_prompt, CAPS)

In [19]:
import random

def least_to_most_one_shot(sentences, target_label, val_dataset=val_dataset, seed=42, k=1):
    random.seed(seed)
    examples = random.sample(list(val_dataset), k)

    # Format each example
    example_blocks = []
    for ex in examples:
        ex_sents = ex["source_sentences"]
        # Which labels to use
        gold_indices = [i+1 for i, lab in enumerate(ex[f"{target_label}_labels"]) if lab == 1]
        numbered_ex = [f"Sentence {i+1}: {s}" for i, s in enumerate(ex_sents)]
        block = f'''Input:
{"\n".join(numbered_ex)}

Selected indices: {gold_indices}'''
        example_blocks.append(block)

    # The "new task" input
    numbered_input = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]

    return f'''You are an expert in extractive summarization. Your task is to select sentences that express the "{target_label}" aspect of the document.

Aspect definitions:
- challenge: problem, gap, limitation, unmet need, difficulty/motivation.
- approach: method, model, system, algorithm, dataset design, procedure.
- outcome: results, findings, improvements, metrics, performance, impact.

Example(s) (from validation set):
{"\n\n".join(example_blocks)}

---

New task:
Input:
{"\n".join(numbered_input)}

Instructions:
1. First, consider the overall purpose of the document and how each sentence contributes to it.
2. Then, select ONLY sentences that primarily express the "{target_label}" aspect.
3. If none match, return an empty list.
4. Indices are 1-based.
5. Return ONLY valid JSON. Do not include explanations or reasoning in your final output.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''


In [20]:
least_to_most_one_shot_capped = make_capped_renderer(least_to_most_one_shot, CAPS)

**tool augmented**: Each sentence is accompanied by a precomputed importance score (e.g., TF-IDF, BERTScore, LexRank), guiding selection.

In [21]:
def tool_augmented_prompt(sentences, target_label):
    numbered = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]
    input_text = "\n".join(numbered)

    return f'''You are an expert in extractive summarization. Your task is to select sentences that express the "{target_label}" aspect of the document.

Aspect definitions:
- challenge: problem, gap, limitation, unmet need, difficulty/motivation.
- approach: method, model, system, algorithm, dataset design, procedure.
- outcome: results, findings, improvements, metrics, performance, impact.

Input:
{input_text}

Instructions:
1. For each sentence, use the internal `check_aspect(sentence, aspect)` tool.
2. The tool's output is 'match' if the sentence primarily describes the "{target_label}" aspect, otherwise it is 'no_match'.
3. List the sentences that result in a 'match'.
4. If no sentences match, return an empty list.
5. Indices are 1-based.
6. Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''

In [22]:
tool_augmented_capped = make_capped_renderer(tool_augmented_prompt, CAPS)

In [23]:
def tool_augmented_one_shot(sentences, target_label, val_dataset=val_dataset, seed=42):
    assert target_label in ASPECTS, f"target_label must be one of {ASPECTS}"

    rng = random.Random(seed)

    # --- pick a single validation example that has at least one positive for this aspect ---
    idxs = list(range(len(val_dataset)))
    rng.shuffle(idxs)
    chosen = None
    for i in idxs:
        ex = val_dataset[i]
        labs = ex[f"{target_label}_labels"]
        if any(int(v) == 1 for v in labs):
            chosen = ex
            break
    if chosen is None:
        # fallback: just take the first one
        chosen = val_dataset[0]

    ex_sents = chosen["source_sentences"]
    ex_labs  = [int(v) for v in chosen[f"{target_label}_labels"]]
    ex_gold  = [j+1 for j, v in enumerate(ex_labs) if v == 1]

    # --- format the example input as numbered sentences ---
    numbered_example = [f"Sentence {i+1}: {s}" for i, s in enumerate(ex_sents)]
    # Create a synthetic "tool trace" where the tool says 'match' iff gold==1
    tool_trace_lines = []
    for i, s in enumerate(ex_sents):
        verdict = "match" if ex_labs[i] == 1 else "no_match"
        # keep the sentence text in the trace to model the behavior (can be long; OK for one-shot)
        tool_trace_lines.append(
            f'check_aspect("Sentence {i+1}: {s}", "{target_label}") -> "{verdict}"'
        )

    # --- the new task's numbered input ---
    numbered_input = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]

    # --- assemble the prompt ---
    return f'''You are an expert in extractive summarization. Your task is to select sentences that express the "{target_label}" aspect of the document.

Aspect definitions:
- challenge: problem, gap, limitation, unmet need, difficulty/motivation.
- approach: method, model, system, algorithm, dataset design, procedure.
- outcome: results, findings, improvements, metrics, performance, impact.

Example (one-shot from the validation set; fixed seed):
Input:
{"\n".join(numbered_example)}

Thinking Process (Tool Use; do not include in final output):
For each sentence, call the internal tool:
{chr(10).join(tool_trace_lines)}

From the tool results above, list the indices that returned "match".
Selected indices:
{ex_gold}

---

New task:
Input:
{"\n".join(numbered_input)}

Instructions:
1) For each sentence, use the internal `check_aspect(sentence, aspect)` tool.
2) The tool returns "match" if the sentence primarily expresses the "{target_label}" aspect; otherwise "no_match".
3) Collect the indices (1-based) of all sentences that return "match".
4) If none match, return an empty list.
5) Return EXACTLY one JSON object on a single line, with no extra text.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''


In [24]:
tool_augmented_one_shot_capped = make_capped_renderer(tool_augmented_one_shot, CAPS)

**scoring based**: The model assigns scores to each sentence (e.g., 1–5 or 1–10) and selects those with the highest ratings.

In [25]:
def scoring_based_prompt(sentences, target_label):
    numbered = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]
    input_text = "\n".join(numbered)

    return f'''You are an expert in extractive summarization. Your task is to select sentences that express the "{target_label}" aspect of the document.

Aspect definitions:
- challenge: problem, gap, limitation, unmet need, difficulty/motivation.
- approach: method, model, system, algorithm, dataset design, procedure.
- outcome: results, findings, improvements, metrics, performance, impact.

Input:
{input_text}

Instructions:
1. For each sentence, assign a score from 1 (low relevance) to 5 (high relevance) for how well it expresses the "{target_label}" aspect.
2. Only select sentences with a score of 4 or 5.
3. If no sentences meet the threshold, return an empty list.
4. Indices are 1-based.
5. Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''

In [26]:
scoring_based_capped = make_capped_renderer(scoring_based_prompt, CAPS)

In [27]:
import random

ASPECTS = ["challenge", "approach", "outcome"]

def scoring_based_one_shot(sentences, target_label, val_dataset=val_dataset, seed=42):
    assert target_label in ASPECTS, f"target_label must be one of {ASPECTS}"

    rng = random.Random(seed)

    # --- choose one validation example that has at least one positive for this aspect ---
    idxs = list(range(len(val_dataset)))
    rng.shuffle(idxs)
    chosen = None
    for i in idxs:
        ex = val_dataset[i]
        labs = ex[f"{target_label}_labels"]
        if any(int(v) == 1 for v in labs):
            chosen = ex
            break
    if chosen is None:
        chosen = val_dataset[0]

    ex_sents = chosen["source_sentences"]
    ex_labs  = [int(v) for v in chosen[f"{target_label}_labels"]]
    ex_gold  = [j+1 for j, v in enumerate(ex_labs) if v == 1]

    # --- fake scoring scheme: high score (4–5) if gold label=1, else low (1–2) ---
    example_scores = [5 if v == 1 else 2 for v in ex_labs]

    numbered_example = [f"Sentence {i+1}: {s}" for i, s in enumerate(ex_sents)]
    numbered_input   = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]

    return f'''You are an expert in extractive summarization. Your task is to select sentences that express the "{target_label}" aspect of the document.

Aspect definitions:
- challenge: problem, gap, limitation, unmet need, difficulty/motivation.
- approach: method, model, system, algorithm, dataset design, procedure.
- outcome: results, findings, improvements, metrics, performance, impact.

Example (one-shot from validation set; fixed seed):
Input:
{"\n".join(numbered_example)}

Scoring process (illustrative, not to be returned):
For the target aspect "{target_label}", assign a score 1–5 to each sentence for aspect relevance:
{"; ".join([f"Sentence {i+1} → {sc}" for i, sc in enumerate(example_scores)])}

Select sentences with score ≥ 4:
Selected indices: {ex_gold}

---

New task:
Input:
{"\n".join(numbered_input)}

Instructions:
1. For each sentence, assign a score from 1 (low relevance) to 5 (high relevance) for how well it expresses the "{target_label}" aspect.
2. Only select sentences with a score of 4 or 5.
3. If no sentences meet the threshold, return an empty list.
4. Indices are 1-based.
5. Return ONLY valid JSON. Do not include explanations or the scores in your final output.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''


In [28]:
scoring_based_one_shot_capped = make_capped_renderer(scoring_based_one_shot, CAPS)

**self ask**: The model generates internal questions like “What is this document about?” and selects the sentences that answer them.

In [29]:
def self_ask_prompt(sentences, target_label):
    numbered = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]
    input_text = "\n".join(numbered)

    return f'''You are an expert in extractive summarization. Your task is to select sentences that express the "{target_label}" aspect of the document.

Aspect definitions:
- challenge: problem, gap, limitation, unmet need, difficulty/motivation.
- approach: method, model, system, algorithm, dataset design, procedure.
- outcome: results, findings, improvements, metrics, performance, impact.

Input:
{input_text}

Instructions:
1. Reason step-by-step. For each sentence, ask the question: "Does this sentence primarily express the "{target_label}" aspect?"
2. Answer the question with "Yes" or "No".
3. Compile a list of all sentences for which the answer was "Yes".
4. If no sentences meet the criteria, return an empty list.
5. Indices are 1-based.
6. Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''

In [30]:
self_ask_capped = make_capped_renderer(self_ask_prompt, CAPS)

In [31]:
import random

ASPECTS = ["challenge", "approach", "outcome"]

def self_ask_one_shot(sentences, target_label, val_dataset=val_dataset, seed=42):
    assert target_label in ASPECTS, f"target_label must be one of {ASPECTS}"

    rng = random.Random(seed)

    # --- pick a validation example that has at least one positive for this aspect ---
    idxs = list(range(len(val_dataset)))
    rng.shuffle(idxs)
    chosen = None
    for i in idxs:
        ex = val_dataset[i]
        labs = ex[f"{target_label}_labels"]
        if any(int(v) == 1 for v in labs):
            chosen = ex
            break
    if chosen is None:
        chosen = val_dataset[0]

    ex_sents = chosen["source_sentences"]
    ex_labs  = [int(v) for v in chosen[f"{target_label}_labels"]]
    ex_gold  = [j+1 for j, v in enumerate(ex_labs) if v == 1]

    # --- construct Yes/No decisions for the example ---
    example_decisions = ["Yes" if v == 1 else "No" for v in ex_labs]

    numbered_example = [f"Sentence {i+1}: {s}" for i, s in enumerate(ex_sents)]
    numbered_input   = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]

    return f'''You are an expert in extractive summarization. Your task is to select sentences that express the "{target_label}" aspect of the document.

Aspect definitions:
- challenge: problem, gap, limitation, unmet need, difficulty/motivation.
- approach: method, model, system, algorithm, dataset design, procedure.
- outcome: results, findings, improvements, metrics, performance, impact.

Example (one-shot from validation set; fixed seed):
Input:
{"\n".join(numbered_example)}

Decision log (concise, not to be returned):
For the target aspect "{target_label}", answer Yes/No for each sentence:
{"; ".join([f"Sentence {i+1} → {d}" for i, d in enumerate(example_decisions)])}

Selected indices (Yes answers): {ex_gold}

---

New task:
Input:
{"\n".join(numbered_input)}

Instructions:
1. For each sentence, ask: "Does this sentence primarily express the '{target_label}' aspect?"
2. Answer each with "Yes" or "No" (internally).
3. Select all sentences answered "Yes".
4. If none, return an empty list.
5. Indices are 1-based.
6. Return ONLY valid JSON. Do not include the questions, answers, or any explanations in the final output.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''


In [32]:
self_ask_one_shot_capped = make_capped_renderer(self_ask_one_shot, CAPS)

- Transform gold standard to match output for each label:

  so for example if the challenge_labels looks like this:

  `[1,0,0,1,1,0]`

  it will become

  `[1, 4, 5]`

- helper functions

In [33]:
import json, re

def safe_extract_json_strict(text: str):
    text = text.strip()

    try:
        js = json.loads(text)
        if isinstance(js, dict) and "selected_sentences" in js:
            return js
        if isinstance(js, list):
            return {"selected_sentences": js}
    except Exception:
        pass

    text_clean = re.sub(r"^```[\w-]*\s*\n", "", text, flags=re.S)
    text_clean = re.sub(r"\n```$", "", text_clean, flags=re.S).strip()

    objs = re.findall(r"\{[\s\S]*?\}", text_clean)
    for s in reversed(objs):
        try:
            js = json.loads(s)
            if isinstance(js, dict) and "selected_sentences" in js:
                return js
        except Exception:
            continue

    m = re.search(r"(?m)^\s*\[(?:\s*\d+\s*(?:,\s*\d+\s*)*)?\]\s*$", text_clean)
    if m:
        try:
            arr = json.loads(m.group(0))
            return {"selected_sentences": arr}
        except Exception:
            pass

    return {}


- for rouge calculation the abstractive dataset is needed

In [34]:
from datasets import load_dataset
from rouge_score import rouge_scorer
import numpy as np
import pandas as pd

# Load ACLSum splits
extractive_test = dataset
abstractive_test = load_dataset("sobamchan/aclsum", "abstractive")["test"]

# Map: doc id -> {"challenge": str, "approach": str, "outcome": str}
ABS_ASPECTS = ["challenge", "approach", "outcome"]
abs_by_id = {
    ex["id"]: {a: ex[a] for a in ABS_ASPECTS}
    for ex in abstractive_test
}

train.jsonl: 0.00B [00:00, ?B/s]

val.jsonl: 0.00B [00:00, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/50 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

- The qwen model is pretrained on chat-like inputs, and therefore the input we have needs to be translated as a chat as well, and needs a system role

- llama does not use this in the same way as Qwen and the system instruction is typically baked into the user prompt or handled differently

- metric calculation

As requested, the following metrics are calculated to evaluate model's performance:
- precision, recall and F1: they are the most straightforward metrics and they represent in order, the ratio of TP/(TP+FP), TP/(TP+FN) and the harmonic mean of precision and recall. They offer the principal analysis against the gold standard to understand how well the two models perform the predictions.
- ROUGE-1: unigram (word-level) overlap between the predicted summary and the human abstractive summary
- ROUGE-2: bigram overlap, more sensitive to fluency and local coherence.
- ROUGE-L: longest common subsequence, capturing sentence-level structural similarity.

Here, ROUGE provides a secondary perspective: while precision/recall/F1 check if the exact gold extractive sentences were found, ROUGE checks how much the extracted sentences overlap with the human abstractive references. This connects extractive performance to human-oriented summaries.

- Oracle ROUGE (per aspect and union): For each document, we construct an oracle summary by greedily selecting the subset of sentences that maximizes ROUGE with respect to the human abstractive summary. This represents the upper bound of predictions. The Oracle gap (RL) is the difference between model ROUGE-L and oracle ROUGE-L. A large gap means the model is far from the extractive upper bound, while a small gap means the model is nearly optimal under extractive constraints.

In [35]:
# Oracle (greedy ROUGE-L F1 upper bound)
_oracle_rouge = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)

def build_oracle_indices(sentences, abstr_ref, max_len=None):
    selected, remaining, best_f = [], list(range(len(sentences))), 0.0
    while remaining and (max_len is None or len(selected) < max_len):
        best_pair = (best_f, None)
        for i in remaining:    # remaining in the list of sentences not yet selected
            # all already chosen sentences + new one
            hyp = " ".join([sentences[j] for j in selected + [i]])

            # compute rouge f1
            f = _oracle_rouge.score(abstr_ref, hyp)["rougeL"].fmeasure

            # store pair if they yield a better rouge score than the current one
            if f > best_pair[0]:
                best_pair = (f, i)

        # if no candidate improved it stop
        if best_pair[1] is None:
            break
        # otherwise, add the new best candidate
        best_f, chosen = best_pair
        selected.append(chosen)
        remaining.remove(chosen)

    # return indices of selected sentences
    return [i+1 for i in selected]  # 1-based

# Exact-match metrics on indices
def calculate_metrics(gold_indices, pred_indices):
    gs, ps = set(gold_indices), set(pred_indices)

    # true positive, false positive, false negatives
    tp = len(gs & ps); fp = len(ps - gs); fn = len(gs - ps)

    # precision
    p = tp/(tp+fp) if (tp+fp) else 0.0
    # recall
    r = tp/(tp+fn) if (tp+fn) else 0.0
    # f1
    f = 2*p*r/(p+r) if (p+r) else 0.0
    return {"precision": p, "recall": r, "f1_score": f, "tp": tp, "fp": fp, "fn": fn}

# # for union calculation
# def _concat_abs_refs(abs_ref):
#     parts = []
#     for k in ("challenge", "approach", "outcome"):
#         if abs_ref and abs_ref.get(k):
#             parts.append(abs_ref[k])
#     return " ".join(parts)

- evaluation loop

This is the core of the project. The function takes as input the dataset, model (model name and tokenizer) as well as the prompt technique that needs to be used, the truth label for the whole dataset, the aspects (challenge, approach, outcome) and the truth labels taken from the abstractive dataset for the calculation of rouge scores. The following inputs are a series of flags which indicate how many documents to analyze, regulates the prints inside the function, the max length of the oracle and finally if the capping explained before is applied or not to enforce a max length of elements in the output array.

The goal of the function is to call the correct prediction method (with either Llama or Qwen) and compute the metrics which underline the quality of the results.

The problem is tackled in two ways:

- *aspect-based*: compute metrics according to each aspect, to understand if there are some which are more challenging compared to others.

- *aspect-agnostic*: compute the metrics according to all aspects together to understand the general prediction power of the model. This is done through union (which aggregates the predicted values and gold standard before calculating the metrics on that), and overall, which instead takes the final values of all aspects (+union) and calculates the mean to have a good understanding of model performance.

Both these aspects are necessary to get a comprehensive view of performance.

In [36]:
from rouge_score import rouge_scorer
import numpy as np
import torch

# build aspect prompts together instead of one by one for speed
def _build_aspect_batch(tokenizer, prompt_fn, sentences, aspects, system_msg=None):
    prompts = []
    # loop thorough aspects
    for a in aspects:
        # build prompt
        user = prompt_fn(sentences, a)
        messages = ([{"role": "system", "content": system_msg}] if system_msg else []) + [
            {"role": "user", "content": user}
        ]
        # build chat-like structure
        chat = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        prompts.append(chat)
    # tokenize
    toks = tokenizer(prompts, return_tensors="pt", padding=True)
    prompt_lens = toks["attention_mask"].sum(dim=1)  # tensor per sample
    return toks, prompt_lens

def run_evaluation_with_rouge_and_oracle(
    dataset,
    model_name,
    model,
    tokenizer,
    prompt_technique,
    gold_all,
    aspects,
    abs_by_id,
    start=0,
    stop=None,
    verbose=False,
    oracle_max_len=None,
    capped=False,
):
    """
    Returns dict with:
      - technique, model_name
      - per_aspect (includes 'union')
      - overall_macro
      - per_doc rows (ready for DataFrame)
    """
    if stop is None:
        stop = len(dataset)
    stop = min(stop, len(dataset))

    # concat three abstractive refs for union
    def _concat_abs_refs(abs_ref):
        parts = []
        for k in ("challenge", "approach", "outcome"):
            if abs_ref and abs_ref.get(k):
                parts.append(abs_ref[k])
        return " ".join(parts)

    # aggregators
    agg = {a: {"P": [], "R": [], "F1": []} for a in aspects}
    rouge_sc = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    r_model = {a: {"r1": [], "r2": [], "rL": []} for a in aspects}
    r_oracl = {a: {"r1": [], "r2": [], "rL": []} for a in aspects}
    r_gap   = {a: {"rL_gap": []} for a in aspects}
    per_doc_rows = []

    union_key = "union"
    union_initialized = False

    # short, greedy outputs are enough (JSON)
    GEN_KW = dict(max_new_tokens=64, do_sample=False, use_cache=True)

    for doc_idx in range(start, stop):
        ex = dataset[doc_idx]
        doc_id    = ex["id"]
        sentences = ex["source_sentences"]
        gold_std  = gold_all[doc_idx]
        abs_ref   = abs_by_id.get(doc_id, None)

        if verbose:
            print("="*80)
            print(f"Doc {doc_idx} (id={doc_id}) | #sentences={len(sentences)}")
            print("-"*80)

        # ---- PER-ASPECT: do ONE generate() for all aspects of this doc
        is_qwen = ("Qwen" in model_name)
        system_msg = "You are an expert in extractive summarization." if is_qwen else None

        toks, prompt_lens = _build_aspect_batch(
            tokenizer=tokenizer,
            prompt_fn=prompt_technique,
            sentences=sentences,
            aspects=aspects,
            system_msg=system_msg,
        )
        toks = {k: v.to(model.device, non_blocking=True) for k, v in toks.items()}

        with torch.inference_mode():
            out_ids = model.generate(
                **toks,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
                **GEN_KW
            )

        # decode tails & parse JSON -> indices per aspect
        preds_this_doc = {}
        for a, row, plen in zip(aspects, out_ids, prompt_lens):
            tail_ids = row[int(plen):]
            out_str = tokenizer.decode(tail_ids, skip_special_tokens=True)

            js = safe_extract_json_strict(out_str)
            idxs = js.get("selected_sentences", [])

            # sanitize 1-based indices
            n = len(sentences)
            clean = []
            for v in idxs:
                if isinstance(v, (int, float)):
                    v = int(v)
                    if 1 <= v <= n:
                        clean.append(v)
            clean = sorted(set(clean))

            # optional cap
            if capped and "CAPS" in globals():
                capK = CAPS.get(a, None)
                if capK is not None and "_cap_indices_in_order" in globals():
                    clean = _cap_indices_in_order(clean, capK)

            preds_this_doc[a] = clean

        # ---- metrics/rouge/oracle per aspect
        for aspect in aspects:
            pred = preds_this_doc[aspect]
            gold_idx = gold_std.get(aspect, [])

            # exact-match metrics
            m = calculate_metrics(gold_idx, pred)
            agg[aspect]["P"].append(m["precision"])
            agg[aspect]["R"].append(m["recall"])
            agg[aspect]["F1"].append(m["f1_score"])

            # ROUGE vs abstractive + oracle
            r1=r2=rL=0.0
            or_r1=or_r2=or_rL=0.0
            gap=0.0
            oracle_idx=[]

            if abs_ref and abs_ref.get(aspect):
                ref = abs_ref[aspect]
                hyp_model = indices_to_text(sentences, pred)

                rs_m = rouge_sc.score(ref, hyp_model)
                r1, r2, rL = rs_m["rouge1"].fmeasure, rs_m["rouge2"].fmeasure, rs_m["rougeL"].fmeasure
                r_model[aspect]["r1"].append(r1)
                r_model[aspect]["r2"].append(r2)
                r_model[aspect]["rL"].append(rL)

                oracle_idx = build_oracle_indices(sentences, ref, max_len=oracle_max_len)
                hyp_or = indices_to_text(sentences, oracle_idx)
                rs_o = rouge_sc.score(ref, hyp_or)
                or_r1, or_r2, or_rL = rs_o["rouge1"].fmeasure, rs_o["rouge2"].fmeasure, rs_o["rougeL"].fmeasure
                r_oracl[aspect]["r1"].append(or_r1)
                r_oracl[aspect]["r2"].append(or_r2)
                r_oracl[aspect]["rL"].append(or_rL)
                gap = or_rL - rL
                r_gap[aspect]["rL_gap"].append(gap)

            if verbose:
                print(f"  -> {aspect.capitalize():9s}")
                print(f"     Predicted: {pred}")
                print(f"     Gold:      {sorted(gold_idx)}")
                print(f"     Metrics:   P={m['precision']:.2f} R={m['recall']:.2f} F1={m['f1_score']:.2f}")
                if abs_ref and abs_ref.get(aspect):
                    print(f"     ROUGE F1 (model):  R1={r1:.3f}  R2={r2:.3f}  RL={rL:.3f}")
                    print(f"     ROUGE F1 (oracle): R1={or_r1:.3f} R2={or_r2:.3f} RL={or_rL:.3f}")
                    print(f"     Oracle gap (RL):   {gap:+.3f}")

            per_doc_rows.append({
                "technique": prompt_technique,
                "model_name": model_name,
                "doc_idx": doc_idx,
                "doc_id": doc_id,
                "aspect": aspect,
                "num_sentences": len(sentences),
                "pred_indices": pred,
                "gold_indices": sorted(gold_idx),
                "precision": m["precision"],
                "recall": m["recall"],
                "f1": m["f1_score"],
                "rouge1_model": r1, "rouge2_model": r2, "rougeL_model": rL,
                "rouge1_oracle": or_r1, "rouge2_oracle": or_r2, "rougeL_oracle": or_rL,
                "oracle_gap_rougeL": gap,
                "oracle_indices": oracle_idx,
            })

        # ---- UNION (aspect-agnostic)
        gold_union = sorted(set().union(*[set(gold_std.get(a, [])) for a in aspects]))
        pred_union = sorted(set().union(*[set(preds_this_doc.get(a, [])) for a in aspects]))
        mU = calculate_metrics(gold_union, pred_union)

        if not union_initialized:
            agg[union_key] = {"P": [], "R": [], "F1": []}
            r_model[union_key] = {"r1": [], "r2": [], "rL": []}
            r_oracl[union_key] = {"r1": [], "r2": [], "rL": []}
            r_gap[union_key]   = {"rL_gap": []}
            union_initialized = True

        agg[union_key]["P"].append(mU["precision"])
        agg[union_key]["R"].append(mU["recall"])
        agg[union_key]["F1"].append(mU["f1_score"])

        r1U=r2U=rLU=0.0
        or_r1U=or_r2U=or_rLU=0.0
        gapU=0.0
        oracle_union_idx=[]

        if abs_ref:
            refU = _concat_abs_refs(abs_ref)
            hypU = indices_to_text(sentences, pred_union)
            rsU  = rouge_sc.score(refU, hypU)
            r1U, r2U, rLU = rsU["rouge1"].fmeasure, rsU["rouge2"].fmeasure, rsU["rougeL"].fmeasure
            r_model[union_key]["r1"].append(r1U)
            r_model[union_key]["r2"].append(r2U)
            r_model[union_key]["rL"].append(rLU)

            oracle_union_idx = build_oracle_indices(sentences, refU, max_len=oracle_max_len)
            hyp_orU = indices_to_text(sentences, oracle_union_idx)
            rs_orU  = rouge_sc.score(refU, hyp_orU)
            or_r1U, or_r2U, or_rLU = rs_orU["rouge1"].fmeasure, rs_orU["rouge2"].fmeasure, rs_orU["rougeL"].fmeasure
            r_oracl[union_key]["r1"].append(or_r1U)
            r_oracl[union_key]["r2"].append(or_r2U)
            r_oracl[union_key]["rL"].append(or_rLU)
            gapU = or_rLU - rLU
            r_gap[union_key]["rL_gap"].append(gapU)

        if verbose:
            print(f"  -> {'Union':9s}")
            print(f"     Predicted: {pred_union}")
            print(f"     Gold:      {gold_union}")
            print(f"     Metrics:   P={mU['precision']:.2f} R={mU['recall']:.2f} F1={mU['f1_score']:.2f}")
            if abs_ref:
                print(f"     ROUGE F1 (model):  R1={r1U:.3f}  R2={r2U:.3f}  RL={rLU:.3f}")
                print(f"     ROUGE F1 (oracle): R1={or_r1U:.3f} R2={or_r2U:.3f} RL={or_rLU:.3f}")
                print(f"     Oracle gap (RL):   {gapU:+.3f}")

        per_doc_rows.append({
            "technique": prompt_technique,
            "model_name": model_name,
            "doc_idx": doc_idx,
            "doc_id": doc_id,
            "aspect": "union",
            "num_sentences": len(sentences),
            "pred_indices": pred_union,
            "gold_indices": gold_union,
            "precision": mU["precision"],
            "recall": mU["recall"],
            "f1": mU["f1_score"],
            "rouge1_model": r1U, "rouge2_model": r2U, "rougeL_model": rLU,
            "rouge1_oracle": or_r1U, "rouge2_oracle": or_r2U, "rougeL_oracle": or_rLU,
            "oracle_gap_rougeL": gapU,
            "oracle_indices": oracle_union_idx,
        })

        if verbose:
            print()

    # ---- aggregate (per aspect + union + overall)
    all_aspects = aspects + (["union"] if union_initialized else [])

    per_aspect_summary = {}
    for a in all_aspects:
        P,R,F = agg[a]["P"], agg[a]["R"], agg[a]["F1"]
        r1m,r2m,rLm = r_model[a]["r1"], r_model[a]["r2"], r_model[a]["rL"]
        r1o,r2o,rLo = r_oracl[a]["r1"], r_oracl[a]["r2"], r_oracl[a]["rL"]
        gaps = r_gap[a]["rL_gap"]
        per_aspect_summary[a] = {
            "precision": float(np.mean(P)) if P else 0.0,
            "recall":    float(np.mean(R)) if R else 0.0,
            "f1":        float(np.mean(F)) if F else 0.0,
            "rouge1_model": float(np.mean(r1m)) if r1m else 0.0,
            "rouge2_model": float(np.mean(r2m)) if r2m else 0.0,
            "rougeL_model": float(np.mean(rLm)) if rLm else 0.0,
            "rouge1_oracle": float(np.mean(r1o)) if r1o else 0.0,
            "rouge2_oracle": float(np.mean(r2o)) if r2o else 0.0,
            "rougeL_oracle": float(np.mean(rLo)) if rLo else 0.0,
            "oracle_gap_rougeL": float(np.mean(gaps)) if gaps else 0.0,
        }

    overall_macro = {
        k: float(np.mean([per_aspect_summary[a][k] for a in all_aspects])) if all_aspects else 0.0
        for k in ["precision","recall","f1","rouge1_model","rouge2_model","rougeL_model",
                  "rouge1_oracle","rouge2_oracle","rougeL_oracle","oracle_gap_rougeL"]
    }

    return {
        "technique": prompt_technique,
        "model_name": model_name,
        "per_aspect": per_aspect_summary,
        "overall_macro": overall_macro,
        "per_doc": per_doc_rows,
    }


- helper to transform the results into dataframes

In [37]:
def results_to_dataframe(results_list):
    rows_doc = []
    for res in results_list:
        rows_doc.extend(res["per_doc"])
    df_docs = pd.DataFrame(rows_doc)

    rows_sum = []
    for res in results_list:
        tech = res["technique"]; model = res["model_name"]
        row = {"technique": tech, "model_name": model}
        for a, vals in res["per_aspect"].items():
            for k, v in vals.items():
                row[f"{a}_{k}"] = v
        for k, v in res["overall_macro"].items():
            row[f"overall_{k}"] = v
        rows_sum.append(row)
    df_summary = pd.DataFrame(rows_sum)
    return df_docs, df_summary

- helper to clean said dataframes

In [38]:
import re
import pandas as pd
import numpy as np

def _tech_name(x):
    if callable(x):
        return x.__name__
    s = str(x)
    m = re.search(r"<function\s+([A-Za-z0-9_]+)\s+at\s+0x[0-9a-fA-F]+>", s)
    return m.group(1) if m else s

def make_three_aspect_tables(
    df_summary: pd.DataFrame,
    aspects=("challenge","approach","outcome", "union", "overall"),
    include_metrics=None,
    collapse_duplicates="mean",
    multi_model_columns=True,
    clean_tech_names=True,
):
    """
    Returns: (challenge_df, approach_df, outcome_df, union_df, overall_df)
    Each DF has:
      - rows = metrics for that aspect
      - columns = techniques  (or MultiIndex (model_name, technique) if multi_model_columns=True)
    Values are aggregated per technique (and model) if duplicates exist.
    """
    df = df_summary.copy()

    # clean technique column
    if "technique" in df.columns and clean_tech_names:
        df["technique"] = df["technique"].apply(_tech_name)

    # sanity: ensure model_name exists
    if "model_name" not in df.columns:
        df["model_name"] = "model"

    # helper to build one aspect table
    def _one_aspect_table(aspect):
        # pick all columns for this aspect
        asp_cols = [c for c in df.columns if c.startswith(f"{aspect}_")]
        if include_metrics:
            # restrict to requested metrics if provided
            wanted = {f"{aspect}_{m}" for m in include_metrics}
            asp_cols = [c for c in asp_cols if c in wanted]
        if not asp_cols:
            return pd.DataFrame()

        # melt to long: one row per (run, metric)
        long = df[["model_name","technique"] + asp_cols].melt(
            id_vars=["model_name","technique"],
            value_vars=asp_cols,
            var_name="metric_full",
            value_name="value",
        )
        # strip aspect_ prefix -> metric name only
        long["metric"] = long["metric_full"].str.replace(f"^{aspect}_", "", regex=True)
        long = long.drop(columns=["metric_full"])

        # (optional) collapse duplicates per (model, technique, metric)
        if collapse_duplicates:
            agg_fn = {"mean": np.mean, "max": np.max, "min": np.min}.get(collapse_duplicates, np.mean)
            long = (long
                    .groupby(["model_name","technique","metric"], as_index=False)["value"]
                    .aggregate(agg_fn))

        # pivot: rows=metric, cols=technique (or MultiIndex)
        if multi_model_columns and df["model_name"].nunique() > 1:
            wide = long.pivot_table(index="metric", columns=["model_name","technique"], values="value")
        else:
            wide = long.pivot_table(index="metric", columns="technique", values="value")

        # sort metrics nicely (precision, recall, f1 first)
        order_front = ["precision","recall","f1"]
        rest = [m for m in wide.index if m not in order_front]
        wide = wide.loc[[m for m in order_front if m in wide.index] + rest]

        # nice formatting
        wide = wide.sort_index(axis=1)
        return wide

    tables = []
    for a in aspects:
        tables.append(_one_aspect_table(a))

    # return in requested order: (challenge_df, approach_df, outcome_df)
    return tuple(tables)


- running the loops

In [39]:
techniques = [vanilla_prompt, least_to_most_prompt, least_to_most_one_shot,
              tool_augmented_prompt, tool_augmented_one_shot,
              self_ask_prompt, self_ask_one_shot,
              scoring_based_prompt, scoring_based_one_shot]

techniques_str = ["vanilla_prompt", "least_to_most_prompt", "least_to_most_one_shot",
              "tool_augmented_prompt", "tool_augmented_one_shot",
              "self_ask_prompt", "self_ask_one_shot",
              "scoring_based_prompt", "scoring_based_one_shot"]

techniques_capped = [vanilla_capped, least_to_most_capped, least_to_most_one_shot_capped, tool_augmented_capped, tool_augmented_one_shot_capped,
                     self_ask_capped, self_ask_one_shot_capped, scoring_based_capped, scoring_based_one_shot_capped]

techniques_capped_str = ["vanilla_capped", "least_to_most_capped", "least_to_most_one_shot_capped", "tool_augmented_capped", "tool_augmented_one_shot_capped",
                     "self_ask_capped", "self_ask_one_shot_capped", "scoring_based_capped", "scoring_based_one_shot_capped"]

- helper function to load the models in a way which improves speed

In [40]:
def load_model_and_tokenizer(model_name: str):
    kwargs = dict(
        torch_dtype=torch.bfloat16,          # BF16 = faster & less memory
        device_map="auto",                   # automatically place on GPU
    )

    if "Qwen" in model_name:
        tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            **kwargs
        )
        print("Loaded Qwen model and tokenizer (with BF16).")

    elif "Llama" in model_name:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            **kwargs
        )
        print("Loaded Llama model and tokenizer (with BF16).")

    else:
        raise ValueError("Unsupported model name. Please use a Qwen or Llama model.")

    if tokenizer.pad_token_id is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"  # good for batching

    return model.eval(), tokenizer


### Llama

In [ ]:
model_name = "meta-llama/Llama-3.2-3B-Instruct" # Or "meta-llama/Llama-3.2-3B-Instruct" || Qwen/Qwen3-4B

model, tokenizer = load_model_and_tokenizer(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded Llama model and tokenizer (with BF16).


In [ ]:
results = []
for tech in techniques:
    print(f"Model Name: {model_name} | Technique: {tech}")
    res = run_evaluation_with_rouge_and_oracle(
        dataset=dataset,
        model_name=model_name,
        model=model,
        tokenizer=tokenizer,
        prompt_technique=tech,
        gold_all=gold_all,
        aspects=ASPECTS,
        abs_by_id=abs_by_id,
        start=0,
        stop=None,
        verbose=True,
        oracle_max_len=None,
        capped = False
    )
    results.append(res)

df_docs, df_summary = results_to_dataframe(results)

Streaming output truncated to the last 5000 lines.
     Predicted: [2, 3, 5, 8, 9, 10, 12, 13, 14, 15]
     Gold:      [1, 2, 3, 7, 8, 9, 10, 13, 14]
     Metrics:   P=0.70 R=0.78 F1=0.74
     ROUGE F1 (model):  R1=0.281  R2=0.036  RL=0.180
     ROUGE F1 (oracle): R1=0.467 R2=0.038 RL=0.243
     Oracle gap (RL):   +0.063

Doc 44 (id=E06-1015) | #sentences=35
--------------------------------------------------------------------------------
  -> Challenge
     Predicted: [2, 9, 14, 16, 17, 23, 33]
     Gold:      [1, 2, 8, 9, 33]
     Metrics:   P=0.43 R=0.60 F1=0.50
     ROUGE F1 (model):  R1=0.144  R2=0.058  RL=0.077
     ROUGE F1 (oracle): R1=0.466 R2=0.338 RL=0.466
     Oracle gap (RL):   +0.389
  -> Approach 
     Predicted: []
     Gold:      [3, 19]
     Metrics:   P=0.00 R=0.00 F1=0.00
     ROUGE F1 (model):  R1=0.000  R2=0.000  RL=0.000
     ROUGE F1 (oracle): R1=0.483 R2=0.321 RL=0.448
     Oracle gap (RL):   +0.448
  -> Outcome  
     Predicted: [4, 25, 26, 27, 30, 33, 34, 35]


In [ ]:
challenge_df, approach_df, outcome_df, union_df, overall_df = make_three_aspect_tables(
    df_summary,
    include_metrics=["precision","recall","f1","rouge1_model","rouge2_model","rougeL_model",
                     "rouge1_oracle","rouge2_oracle","rougeL_oracle","oracle_gap_rougeL"],
    multi_model_columns=False,
)


/tmp/ipython-input-2363513484.py:65: FutureWarning: The provided callable <function mean at 0x7a6b90b2c9a0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .aggregate(agg_fn))
/tmp/ipython-input-2363513484.py:65: FutureWarning: The provided callable <function mean at 0x7a6b90b2c9a0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .aggregate(agg_fn))
/tmp/ipython-input-2363513484.py:65: FutureWarning: The provided callable <function mean at 0x7a6b90b2c9a0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .aggregate(agg_fn))
/tmp/ipython-input-2363513484.py:65: FutureWarning: The provided callable <function mean at 0x

There is a very interesting finding taken from this. When the prediction method is changed in favor of speed-oriented batching, the empirical behavior changes, even though the prompts are the same.

In [ ]:
challenge_df

technique,least_to_most_one_shot,least_to_most_prompt,scoring_based_one_shot,scoring_based_prompt,self_ask_one_shot,self_ask_prompt,tool_augmented_one_shot,tool_augmented_prompt,vanilla_prompt
metric,,,,,,,,,
precision,0.262193,0.131504,0.104671,0.128817,0.042201,0.085448,0.089949,0.063833,0.159508
recall,0.310500,0.235845,0.147083,0.265143,0.127429,0.160944,0.147381,0.101214,0.354615
f1,0.259903,0.157346,0.106003,0.161114,0.058432,0.104363,0.091633,0.073726,0.203356
oracle_gap_rougeL,0.258833,0.319242,0.301109,0.317218,0.330527,0.338287,0.307382,0.344370,0.302741
rouge1_model,0.160373,0.074690,0.103805,0.076492,0.060063,0.049379,0.094831,0.038103,0.098095
rouge1_oracle,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003
rouge2_model,0.058235,0.029581,0.024340,0.029416,0.016816,0.017969,0.021013,0.014749,0.039873
rouge2_oracle,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200
rougeL_model,0.112593,0.052184,0.070317,0.054208,0.040899,0.033139,0.064044,0.027056,0.068685


In [ ]:
approach_df

technique,least_to_most_one_shot,least_to_most_prompt,scoring_based_one_shot,scoring_based_prompt,self_ask_one_shot,self_ask_prompt,tool_augmented_one_shot,tool_augmented_prompt,vanilla_prompt
metric,,,,,,,,,
precision,0.303144,0.115511,0.282167,0.140318,0.208630,0.062036,0.254468,0.053306,0.167474
recall,0.329245,0.176507,0.280102,0.218766,0.227682,0.075258,0.339075,0.068897,0.300630
f1,0.256825,0.132011,0.246715,0.158286,0.186532,0.062419,0.265861,0.056107,0.205615
oracle_gap_rougeL,0.319371,0.416685,0.311755,0.405910,0.331195,0.429884,0.327066,0.432299,0.399893
rouge1_model,0.167999,0.038300,0.180051,0.052868,0.160380,0.019764,0.161074,0.017116,0.058669
rouge1_oracle,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775
rouge2_model,0.067029,0.019734,0.072380,0.023576,0.052015,0.011209,0.067901,0.008359,0.030002
rouge2_oracle,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252
rougeL_model,0.126807,0.029493,0.134424,0.040268,0.114984,0.016295,0.119113,0.013880,0.046286


In [ ]:
outcome_df

technique,least_to_most_one_shot,least_to_most_prompt,scoring_based_one_shot,scoring_based_prompt,self_ask_one_shot,self_ask_prompt,tool_augmented_one_shot,tool_augmented_prompt,vanilla_prompt
metric,,,,,,,,,
precision,0.325526,0.383223,0.279318,0.364817,0.324712,0.261159,0.319706,0.186897,0.363929
recall,0.384140,0.402595,0.166067,0.470365,0.306013,0.289841,0.338315,0.232127,0.430611
f1,0.318310,0.367014,0.195685,0.382573,0.285740,0.260400,0.302089,0.199763,0.365074
oracle_gap_rougeL,0.284213,0.312387,0.262861,0.320906,0.270906,0.356479,0.278770,0.370575,0.323069
rouge1_model,0.200919,0.151368,0.234216,0.143871,0.221714,0.097211,0.209877,0.075666,0.138743
rouge1_oracle,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727
rouge2_model,0.077998,0.072747,0.066261,0.067372,0.078806,0.044721,0.076815,0.033814,0.069811
rouge2_oracle,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506
rougeL_model,0.144259,0.116084,0.165610,0.107566,0.157565,0.071993,0.149701,0.057897,0.105403


To analyze briefly the results of this run, some things are noticeable:
- it looks like the vanilla prompt does not perform that badly, and methods like the tool augmented with zero shot seems to underperform consistently
- it also looks like the easiest aspect to detect is the outcome, while the hardest is the challenge.

**Why?**:
- Least-to-Most prompting achieved the best average performance, likely because the decomposition step clarifies aspect criteria and curbs over-selection
- Vanilla remained competitive: with clear aspect definitions, small open models exploit strong lexical markers (especially for Approach/Outcome) without the overhead of complex reasoning scaffolds
- for tool-augmented prompting, the extra scaffolding inflates the prompt length without adding real information. For a 3–4B model with limited context processing ability, this can crowd out useful signal from the actual sentences.
- for self-ask prompting techniques, small models tend to not remain consistent in the 30-40 sentence range that we have.
- Scoring-based, like tool-augmented, tends to waste tokens on meta-instructions instead of focusing on the sentence-aspect match.

In [ ]:
union_df

technique,least_to_most_one_shot,least_to_most_prompt,scoring_based_one_shot,scoring_based_prompt,self_ask_one_shot,self_ask_prompt,tool_augmented_one_shot,tool_augmented_prompt,vanilla_prompt
metric,,,,,,,,,
precision,0.542272,0.475530,0.492826,0.477050,0.492249,0.320071,0.487526,0.252067,0.426276
recall,0.487089,0.302986,0.409564,0.368453,0.426365,0.197045,0.519366,0.152508,0.439574
f1,0.473014,0.333031,0.414789,0.380789,0.418985,0.224727,0.477503,0.176857,0.404464
oracle_gap_rougeL,0.189144,0.218913,0.189607,0.217687,0.191969,0.259938,0.205980,0.280346,0.226928
rouge1_model,0.280863,0.230191,0.282890,0.228870,0.276654,0.163861,0.250495,0.126761,0.206674
rouge1_oracle,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235
rouge2_model,0.111570,0.085150,0.104423,0.088069,0.105904,0.058762,0.102346,0.046993,0.086188
rouge2_oracle,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569
rougeL_model,0.166460,0.136691,0.165997,0.137916,0.163635,0.095666,0.149623,0.075257,0.128676


- capped results

In [ ]:
results_capped = []
for tech in techniques_capped:
    print(f"Model Name: {model_name} | Technique: {tech}")
    res = run_evaluation_with_rouge_and_oracle(
        dataset=dataset,
        model_name=model_name,
        model=model,
        tokenizer=tokenizer,
        prompt_technique=tech,
        gold_all=gold_all,
        aspects=ASPECTS,
        abs_by_id=abs_by_id,
        start=0,
        stop=None,
        verbose=True,
        oracle_max_len=None,
        capped = True
    )
    results_capped.append(res)

df_docs_capped, df_summary_capped = results_to_dataframe(results_capped)

Streaming output truncated to the last 5000 lines.
     Predicted: [2, 3, 5, 8, 9, 10, 12, 13, 14, 15]
     Gold:      [1, 2, 3, 7, 8, 9, 10, 13, 14]
     Metrics:   P=0.70 R=0.78 F1=0.74
     ROUGE F1 (model):  R1=0.281  R2=0.036  RL=0.180
     ROUGE F1 (oracle): R1=0.467 R2=0.038 RL=0.243
     Oracle gap (RL):   +0.063

Doc 44 (id=E06-1015) | #sentences=35
--------------------------------------------------------------------------------
  -> Challenge
     Predicted: [2, 9, 14, 16]
     Gold:      [1, 2, 8, 9, 33]
     Metrics:   P=0.50 R=0.40 F1=0.44
     ROUGE F1 (model):  R1=0.207  R2=0.090  RL=0.119
     ROUGE F1 (oracle): R1=0.466 R2=0.338 RL=0.466
     Oracle gap (RL):   +0.347
  -> Approach 
     Predicted: [3, 8, 10, 19, 20, 22]
     Gold:      [3, 19]
     Metrics:   P=0.33 R=1.00 F1=0.50
     ROUGE F1 (model):  R1=0.157  R2=0.084  RL=0.120
     ROUGE F1 (oracle): R1=0.483 R2=0.321 RL=0.448
     Oracle gap (RL):   +0.328
  -> Outcome  
     Predicted: [3, 4, 25, 26]
     Gold

In [49]:
def rename_capped_techniques(df_summary_capped, techniques_capped_str):
    """
    Replace the 'technique' column in df_summary_capped with the provided names,
    in the given order.
    """
    if len(df_summary_capped) != len(techniques_capped_str):
        raise ValueError("Mismatch: dataframe rows != techniques_capped_str length")

    df_summary_capped = df_summary_capped.copy()
    df_summary_capped["technique"] = techniques_capped_str
    return df_summary_capped



In [ ]:
df_summary_capped = rename_capped_techniques(df_summary_capped, techniques_capped_str)

In [ ]:
challenge_df, approach_df, outcome_df, union_df, overall_df = make_three_aspect_tables(
    df_summary_capped,
    include_metrics=["precision","recall","f1","rouge1_model","rouge2_model","rougeL_model",
                     "rouge1_oracle","rouge2_oracle","rougeL_oracle","oracle_gap_rougeL"],
    multi_model_columns=False,
)


/tmp/ipython-input-2363513484.py:65: FutureWarning: The provided callable <function mean at 0x7a6b90b2c9a0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .aggregate(agg_fn))
/tmp/ipython-input-2363513484.py:65: FutureWarning: The provided callable <function mean at 0x7a6b90b2c9a0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .aggregate(agg_fn))
/tmp/ipython-input-2363513484.py:65: FutureWarning: The provided callable <function mean at 0x7a6b90b2c9a0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .aggregate(agg_fn))
/tmp/ipython-input-2363513484.py:65: FutureWarning: The provided callable <function mean at 0x

In [ ]:
challenge_df

technique,least_to_most_capped,least_to_most_one_shot_capped,scoring_based_capped,scoring_based_one_shot_capped,self_ask_capped,self_ask_one_shot_capped,tool_augmented_capped,tool_augmented_one_shot_capped,vanilla_capped
metric,,,,,,,,,
precision,0.410833,0.373333,0.383333,0.153333,0.165000,0.205000,0.080000,0.146667,0.390833
recall,0.429075,0.373139,0.410083,0.172782,0.181571,0.223948,0.073591,0.137615,0.424452
f1,0.393655,0.352212,0.374511,0.147688,0.160644,0.198815,0.070115,0.129668,0.380553
oracle_gap_rougeL,0.216771,0.223274,0.216991,0.257669,0.305533,0.249413,0.348206,0.266789,0.222118
rouge1_model,0.212096,0.214079,0.213991,0.167801,0.090052,0.177821,0.032477,0.160207,0.210733
rouge1_oracle,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003
rouge2_model,0.082977,0.075727,0.081090,0.038653,0.032747,0.045116,0.011457,0.035861,0.077588
rouge2_oracle,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200
rougeL_model,0.154655,0.148152,0.154435,0.113757,0.065893,0.122013,0.023220,0.104636,0.149308


In [ ]:
approach_df

technique,least_to_most_capped,least_to_most_one_shot_capped,scoring_based_capped,scoring_based_one_shot_capped,self_ask_capped,self_ask_one_shot_capped,tool_augmented_capped,tool_augmented_one_shot_capped,vanilla_capped
metric,,,,,,,,,
precision,0.351333,0.432167,0.401333,0.277333,0.129667,0.258500,0.040000,0.260833,0.425833
recall,0.358659,0.376337,0.422357,0.235960,0.124246,0.227248,0.052024,0.214726,0.449742
f1,0.337170,0.372025,0.394158,0.237729,0.121447,0.226420,0.043472,0.224156,0.418228
oracle_gap_rougeL,0.336589,0.277742,0.313221,0.303690,0.404424,0.309847,0.429420,0.301364,0.307275
rouge1_model,0.141066,0.222829,0.173748,0.200106,0.052476,0.192479,0.019587,0.203431,0.184295
rouge1_oracle,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775
rouge2_model,0.066921,0.094299,0.083262,0.067767,0.027843,0.062815,0.010811,0.071130,0.087946
rouge2_oracle,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252
rougeL_model,0.109590,0.168436,0.132958,0.142489,0.041754,0.136331,0.016759,0.144815,0.138904


In [ ]:
outcome_df

technique,least_to_most_capped,least_to_most_one_shot_capped,scoring_based_capped,scoring_based_one_shot_capped,self_ask_capped,self_ask_one_shot_capped,tool_augmented_capped,tool_augmented_one_shot_capped,vanilla_capped
metric,,,,,,,,,
precision,0.559167,0.426667,0.524167,0.308333,0.335833,0.353333,0.135833,0.299167,0.596667
recall,0.511060,0.342628,0.504342,0.180484,0.293588,0.233767,0.135119,0.213004,0.553076
f1,0.506362,0.360269,0.490932,0.215066,0.298855,0.267911,0.130332,0.233342,0.548587
oracle_gap_rougeL,0.247844,0.238061,0.242889,0.256275,0.312965,0.248884,0.378843,0.266258,0.229012
rouge1_model,0.240768,0.262510,0.252993,0.239748,0.155816,0.249422,0.063693,0.228163,0.263351
rouge1_oracle,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727
rouge2_model,0.106594,0.105299,0.111992,0.070868,0.071026,0.086095,0.026330,0.077378,0.119342
rouge2_oracle,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506
rougeL_model,0.180627,0.190410,0.185583,0.172196,0.115507,0.179588,0.049629,0.162213,0.199460


As explained before, capping the results, therefore artificially preventing the model from overpredicting, leads to much better results, especially in terms of precision. Still, it seems like the outcome aspect remains the easiest to predict.

In [ ]:
union_df

technique,least_to_most_capped,least_to_most_one_shot_capped,scoring_based_capped,scoring_based_one_shot_capped,self_ask_capped,self_ask_one_shot_capped,tool_augmented_capped,tool_augmented_one_shot_capped,vanilla_capped
metric,,,,,,,,,
precision,0.592439,0.594192,0.613121,0.448520,0.375755,0.481420,0.206680,0.451312,0.594780
recall,0.465985,0.481972,0.503576,0.345275,0.203722,0.380101,0.107553,0.356773,0.526226
f1,0.504292,0.521889,0.539104,0.382113,0.249661,0.416790,0.133406,0.388942,0.547217
oracle_gap_rougeL,0.184524,0.177093,0.178911,0.194128,0.241814,0.192495,0.291359,0.196662,0.188694
rouge1_model,0.285929,0.300317,0.296613,0.286192,0.201119,0.284603,0.107527,0.279963,0.278471
rouge1_oracle,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235
rouge2_model,0.117021,0.127263,0.126271,0.100253,0.076926,0.103656,0.037277,0.104218,0.119732
rouge2_oracle,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569
rougeL_model,0.171080,0.178511,0.176692,0.161475,0.113789,0.163108,0.064244,0.158941,0.166909


The union dataframe, which represents the outcomes toghether, uniting the gold standard and the prediction, gives ulterior credit to this hypothesis.

### Qwen

In [ ]:
model_name = "Qwen/Qwen3-4B-Instruct-2507" # Or "meta-llama/Llama-3.2-3B-Instruct" || Qwen/Qwen3-4B

model, tokenizer = load_model_and_tokenizer(model_name)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

Loaded Qwen model and tokenizer (with BF16).


In [ ]:
results_qwen = []
for tech in techniques:
    print(f"Model Name: {model_name} | Technique: {tech}")
    res = run_evaluation_with_rouge_and_oracle(
        dataset=dataset,
        model_name=model_name,
        model=model,
        tokenizer=tokenizer,
        prompt_technique=tech,
        gold_all=gold_all,
        aspects=ASPECTS,
        abs_by_id=abs_by_id,
        start=0,
        stop=None,
        verbose=True,
        oracle_max_len=None,
        capped = False
    )
    results_qwen.append(res)

df_docs_qwen, df_summary_qwen = results_to_dataframe(results_qwen)

Streaming output truncated to the last 5000 lines.
     Predicted: [2, 3, 4, 5, 8, 9, 10, 11, 12, 13, 14, 15]
     Gold:      [1, 2, 3, 7, 8, 9, 10, 13, 14]
     Metrics:   P=0.58 R=0.78 F1=0.67
     ROUGE F1 (model):  R1=0.275  R2=0.035  RL=0.172
     ROUGE F1 (oracle): R1=0.467 R2=0.038 RL=0.243
     Oracle gap (RL):   +0.071

Doc 44 (id=E06-1015) | #sentences=35
--------------------------------------------------------------------------------
  -> Challenge
     Predicted: [2, 9, 14, 16, 17, 33]
     Gold:      [1, 2, 8, 9, 33]
     Metrics:   P=0.50 R=0.60 F1=0.55
     ROUGE F1 (model):  R1=0.181  R2=0.073  RL=0.096
     ROUGE F1 (oracle): R1=0.466 R2=0.338 RL=0.466
     Oracle gap (RL):   +0.369
  -> Approach 
     Predicted: [3, 10, 14, 19, 20, 21, 24, 25, 26, 27, 28, 34, 35]
     Gold:      [3, 19]
     Metrics:   P=0.15 R=1.00 F1=0.27
     ROUGE F1 (model):  R1=0.076  R2=0.048  RL=0.071
     ROUGE F1 (oracle): R1=0.483 R2=0.321 RL=0.448
     Oracle gap (RL):   +0.377
  -> Outcom

In [ ]:
df_summary_qwen

,technique,model_name,challenge_precision,challenge_recall,challenge_f1,challenge_rouge1_model,challenge_rouge2_model,challenge_rougeL_model,challenge_rouge1_oracle,challenge_rouge2_oracle,...,overall_precision,overall_recall,overall_f1,overall_rouge1_model,overall_rouge2_model,overall_rougeL_model,overall_rouge1_oracle,overall_rouge2_oracle,overall_rougeL_oracle,overall_oracle_gap_rougeL
0,<function vanilla_prompt at 0x7fa61308b600>,Qwen/Qwen3-4B-Instruct-2507,0.389667,0.620433,0.441521,0.174313,0.072803,0.129088,0.422003,0.2262,...,0.497879,0.672984,0.542371,0.192657,0.092357,0.142164,0.466435,0.253632,0.40042,0.258256
1,<function least_to_most_prompt at 0x7fa61308bd80>,Qwen/Qwen3-4B-Instruct-2507,0.390319,0.604758,0.437734,0.175327,0.072188,0.129334,0.422003,0.2262,...,0.494993,0.649298,0.531882,0.193860,0.091697,0.141644,0.466435,0.253632,0.40042,0.258776
2,<function least_to_most_one_shot at 0x7fa61308...,Qwen/Qwen3-4B-Instruct-2507,0.493366,0.603845,0.491774,0.203864,0.082551,0.151127,0.422003,0.2262,...,0.612119,0.642112,0.593602,0.238326,0.113088,0.174412,0.466435,0.253632,0.40042,0.226008
3,<function tool_augmented_prompt at 0x7fa61308b...,Qwen/Qwen3-4B-Instruct-2507,0.380297,0.564909,0.418809,0.173376,0.068845,0.126856,0.422003,0.2262,...,0.518941,0.648782,0.544458,0.201938,0.094808,0.147450,0.466435,0.253632,0.40042,0.252970
4,<function tool_augmented_one_shot at 0x7fa6130...,Qwen/Qwen3-4B-Instruct-2507,0.503844,0.535540,0.470263,0.221094,0.086675,0.159752,0.422003,0.2262,...,0.618891,0.623981,0.590160,0.243447,0.112136,0.176943,0.466435,0.253632,0.40042,0.223476
5,<function self_ask_prompt at 0x7fa6130a5440>,Qwen/Qwen3-4B-Instruct-2507,0.360422,0.542671,0.396358,0.159327,0.062862,0.116825,0.422003,0.2262,...,0.438747,0.554363,0.461443,0.174487,0.080932,0.126554,0.466435,0.253632,0.40042,0.273865
6,<function self_ask_one_shot at 0x7fa6130a4fe0>,Qwen/Qwen3-4B-Instruct-2507,0.509287,0.528944,0.471966,0.221065,0.084289,0.161488,0.422003,0.2262,...,0.624199,0.625463,0.593670,0.243500,0.111984,0.177079,0.466435,0.253632,0.40042,0.223341
7,<function scoring_based_prompt at 0x7fa613217ba0>,Qwen/Qwen3-4B-Instruct-2507,0.377353,0.533298,0.408837,0.177219,0.071385,0.129145,0.422003,0.2262,...,0.495410,0.604677,0.513132,0.200546,0.092486,0.144800,0.466435,0.253632,0.40042,0.255620
8,<function scoring_based_one_shot at 0x7fa61321...,Qwen/Qwen3-4B-Instruct-2507,0.482413,0.549651,0.469358,0.217224,0.084850,0.156331,0.422003,0.2262,...,0.603409,0.635218,0.587561,0.237842,0.109684,0.171227,0.466435,0.253632,0.40042,0.229192


In [ ]:
challenge_df_q, approach_df_q, outcome_df_q, union_df_q, overall_df_q = make_three_aspect_tables(
    df_summary_qwen,
    include_metrics=["precision","recall","f1","rouge1_model","rouge2_model","rougeL_model",
                     "rouge1_oracle","rouge2_oracle","rougeL_oracle","oracle_gap_rougeL"],
    multi_model_columns=False,
)

/tmp/ipython-input-2401783402.py:64: FutureWarning: The provided callable <function mean at 0x7fa83316c900> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .aggregate(agg_fn))
/tmp/ipython-input-2401783402.py:64: FutureWarning: The provided callable <function mean at 0x7fa83316c900> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .aggregate(agg_fn))
/tmp/ipython-input-2401783402.py:64: FutureWarning: The provided callable <function mean at 0x7fa83316c900> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .aggregate(agg_fn))
/tmp/ipython-input-2401783402.py:64: FutureWarning: The provided callable <function mean at 0x

In [ ]:
challenge_df_q

technique,least_to_most_one_shot,least_to_most_prompt,scoring_based_one_shot,scoring_based_prompt,self_ask_one_shot,self_ask_prompt,tool_augmented_one_shot,tool_augmented_prompt,vanilla_prompt
metric,,,,,,,,,
precision,0.493366,0.390319,0.482413,0.377353,0.509287,0.360422,0.503844,0.380297,0.389667
recall,0.603845,0.604758,0.549651,0.533298,0.528944,0.542671,0.535540,0.564909,0.620433
f1,0.491774,0.437734,0.469358,0.408837,0.471966,0.396358,0.470263,0.418809,0.441521
oracle_gap_rougeL,0.220298,0.242092,0.215095,0.242281,0.209938,0.254601,0.211674,0.244570,0.242338
rouge1_model,0.203864,0.175327,0.217224,0.177219,0.221065,0.159327,0.221094,0.173376,0.174313
rouge1_oracle,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003
rouge2_model,0.082551,0.072188,0.084850,0.071385,0.084289,0.062862,0.086675,0.068845,0.072803
rouge2_oracle,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200
rougeL_model,0.151127,0.129334,0.156331,0.129145,0.161488,0.116825,0.159752,0.126856,0.129088


- Here we notice an improvement in precision and recall compared to the Llama implementation, and recall stays consistently higher compared to precision.

- least-to-most-one-shot and self-ask-one-shot seem to perform better than the others in terms of precision

- vanilla outperforms the other models in terms of recall

- The oracle gap in ROUGE-L is consistently large (~0.21–0.25), meaning there’s still significant room for improvement compared to the extractive oracle, although it remains considerably smaller than the Llama version.



In [ ]:
approach_df_q

technique,least_to_most_one_shot,least_to_most_prompt,scoring_based_one_shot,scoring_based_prompt,self_ask_one_shot,self_ask_prompt,tool_augmented_one_shot,tool_augmented_prompt,vanilla_prompt
metric,,,,,,,,,
precision,0.623737,0.437680,0.567876,0.429034,0.593923,0.360809,0.605967,0.484013,0.477103
recall,0.593021,0.569044,0.651136,0.557790,0.634942,0.473460,0.629607,0.611956,0.647711
f1,0.580923,0.469945,0.579070,0.460419,0.583730,0.387142,0.586680,0.510490,0.520367
oracle_gap_rougeL,0.268557,0.331954,0.293322,0.332759,0.284472,0.355998,0.278736,0.319888,0.322797
rouge1_model,0.220713,0.142242,0.192646,0.142102,0.202751,0.114182,0.209267,0.156927,0.152137
rouge1_oracle,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775
rouge2_model,0.119699,0.078003,0.101382,0.076788,0.108286,0.063096,0.109500,0.085956,0.084230
rouge2_oracle,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252
rougeL_model,0.177622,0.114224,0.152857,0.113419,0.161706,0.090181,0.167442,0.126291,0.123381


- precision is stronger here, consistently with what observed in llama where challenge was very difficult to predict
- recall remains solid with vanilla still outperforming the rest
- The oracle gap is larger here (~0.27–0.36), suggesting approach sentences are harder to capture perfectly.

In [ ]:
outcome_df_q

technique,least_to_most_one_shot,least_to_most_prompt,scoring_based_one_shot,scoring_based_prompt,self_ask_one_shot,self_ask_prompt,tool_augmented_one_shot,tool_augmented_prompt,vanilla_prompt
metric,,,,,,,,,
precision,0.690774,0.570004,0.723091,0.584664,0.748766,0.519396,0.725722,0.619237,0.556671
recall,0.721917,0.751658,0.692089,0.714956,0.699586,0.629511,0.696931,0.747535,0.723241
f1,0.671136,0.613989,0.673262,0.609157,0.692357,0.539553,0.679543,0.639658,0.596985
oracle_gap_rougeL,0.227839,0.257611,0.218736,0.252603,0.211110,0.271606,0.217688,0.245635,0.261838
rouge1_model,0.260695,0.216073,0.274353,0.224635,0.281482,0.199684,0.275032,0.233467,0.210581
rouge1_oracle,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727
rouge2_model,0.127493,0.110265,0.133271,0.111282,0.136753,0.101754,0.133960,0.115838,0.107096
rouge2_oracle,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506
rougeL_model,0.200632,0.170860,0.209736,0.175868,0.217362,0.156865,0.210784,0.182837,0.166634


- consistently to what was observed with the Llama run, the outcome aspect seems to be the easiest to detect, as we can see from the higher precision and recall
- Self_ask_one_shot dominates with the highest precision (0.75) and strong recall (0.70), giving the best F1 (~0.69)
- vanilla still offers high recall and lower precision
- The oracle gap in ROUGE-L remains substantial (~0.21–0.27), showing models can still be pushed further toward the upper bound.

**Overall**:
- Outcome is the easiest aspect: highest absolute scores across precision, recall, and F1. Likely because outcome sentences are more explicit and salient in abstracts.

- Challenge is the hardest: F1s are lower and oracle gaps higher, showing the models struggle to isolate problem statements.

- Approach is in between but with a bigger oracle gap, suggesting it’s a dense/technical aspect where even oracle summaries aren’t easily captured with few sentences.

In [ ]:
union_df_q

technique,least_to_most_one_shot,least_to_most_prompt,scoring_based_one_shot,scoring_based_prompt,self_ask_one_shot,self_ask_prompt,tool_augmented_one_shot,tool_augmented_prompt,vanilla_prompt
metric,,,,,,,,,
precision,0.640598,0.581970,0.640257,0.590590,0.644821,0.514362,0.640030,0.592219,0.568075
recall,0.649666,0.671733,0.647996,0.612666,0.638380,0.571809,0.633846,0.670730,0.700550
f1,0.630574,0.605859,0.628557,0.574116,0.626626,0.522718,0.624153,0.608874,0.610611
oracle_gap_rougeL,0.187336,0.203448,0.189616,0.194835,0.187845,0.213257,0.185808,0.201786,0.206051
rouge1_model,0.268031,0.241796,0.267145,0.258227,0.268702,0.224755,0.268395,0.243983,0.233598
rouge1_oracle,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235
rouge2_model,0.122608,0.106331,0.119234,0.110490,0.118606,0.096015,0.118407,0.108595,0.105301
rouge2_oracle,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569
rougeL_model,0.168267,0.152156,0.165987,0.160768,0.167759,0.142347,0.169796,0.153817,0.149552


- This shows, according to expectations, that Qwen3-4B parameters yields a much better performance also in the aspect-agnostic case. Least to most still performs best, but in this case the second best performing model seems to be the scoring-based. The performances are in general very comparable.

- Vanilla still does not perform badly, especially in terms of recall, where it outperforms all other models

- The oracle gap still suggests that improvements are possible for model performance

In [ ]:
results_qwen_capped = []
for tech in techniques_capped:
    print(f"Model Name: {model_name} | Technique: {tech}")
    res = run_evaluation_with_rouge_and_oracle(
        dataset=dataset,
        model_name=model_name,
        model=model,
        tokenizer=tokenizer,
        prompt_technique=tech,
        gold_all=gold_all,
        aspects=ASPECTS,
        abs_by_id=abs_by_id,
        start=0,
        stop=None,
        verbose=True,
        oracle_max_len=None,
        capped = True
    )
    results_qwen_capped.append(res)

df_docs_qwen_capped, df_summary_qwen_capped = results_to_dataframe(results_qwen_capped)

Streaming output truncated to the last 5000 lines.
     Predicted: [2, 3, 4, 5, 8, 9, 10, 11, 12, 13, 14, 15]
     Gold:      [1, 2, 3, 7, 8, 9, 10, 13, 14]
     Metrics:   P=0.58 R=0.78 F1=0.67
     ROUGE F1 (model):  R1=0.275  R2=0.035  RL=0.172
     ROUGE F1 (oracle): R1=0.467 R2=0.038 RL=0.243
     Oracle gap (RL):   +0.071

Doc 44 (id=E06-1015) | #sentences=35
--------------------------------------------------------------------------------
  -> Challenge
     Predicted: [2, 9, 14, 16]
     Gold:      [1, 2, 8, 9, 33]
     Metrics:   P=0.50 R=0.40 F1=0.44
     ROUGE F1 (model):  R1=0.207  R2=0.090  RL=0.119
     ROUGE F1 (oracle): R1=0.466 R2=0.338 RL=0.466
     Oracle gap (RL):   +0.347
  -> Approach 
     Predicted: [3, 19, 20, 24, 25, 34]
     Gold:      [3, 19]
     Metrics:   P=0.33 R=1.00 F1=0.50
     ROUGE F1 (model):  R1=0.150  R2=0.094  RL=0.121
     ROUGE F1 (oracle): R1=0.483 R2=0.321 RL=0.448
     Oracle gap (RL):   +0.327
  -> Outcome  
     Predicted: [3, 4, 25, 35]
 

In [ ]:
df_summary_qwen_capped

,technique,model_name,challenge_precision,challenge_recall,challenge_f1,challenge_rouge1_model,challenge_rouge2_model,challenge_rougeL_model,challenge_rouge1_oracle,challenge_rouge2_oracle,...,overall_precision,overall_recall,overall_f1,overall_rouge1_model,overall_rouge2_model,overall_rougeL_model,overall_rouge1_oracle,overall_rouge2_oracle,overall_rougeL_oracle,overall_oracle_gap_rougeL
0,<function make_capped_renderer.<locals>.capped...,Qwen/Qwen3-4B-Instruct-2507,0.494167,0.527988,0.480452,0.231795,0.086833,0.167013,0.422003,0.2262,...,0.603712,0.605575,0.580793,0.249070,0.112965,0.178111,0.466435,0.253632,0.40042,0.222308
1,<function make_capped_renderer.<locals>.capped...,Qwen/Qwen3-4B-Instruct-2507,0.475000,0.517655,0.467769,0.225812,0.083070,0.162229,0.422003,0.2262,...,0.600208,0.603177,0.578493,0.247745,0.112574,0.177090,0.466435,0.253632,0.40042,0.223329
2,<function make_capped_renderer.<locals>.capped...,Qwen/Qwen3-4B-Instruct-2507,0.519167,0.562234,0.511057,0.235944,0.094881,0.174245,0.422003,0.2262,...,0.622798,0.612461,0.593935,0.255556,0.118049,0.184534,0.466435,0.253632,0.40042,0.215886
3,<function make_capped_renderer.<locals>.capped...,Qwen/Qwen3-4B-Instruct-2507,0.485000,0.517806,0.475120,0.227994,0.084669,0.163839,0.422003,0.2262,...,0.606327,0.604236,0.582348,0.249896,0.113820,0.179550,0.466435,0.253632,0.40042,0.220870
4,<function make_capped_renderer.<locals>.capped...,Qwen/Qwen3-4B-Instruct-2507,0.472500,0.505849,0.459696,0.227029,0.081337,0.160586,0.422003,0.2262,...,0.622722,0.587775,0.582516,0.258119,0.116017,0.184611,0.466435,0.253632,0.40042,0.215809
5,<function make_capped_renderer.<locals>.capped...,Qwen/Qwen3-4B-Instruct-2507,0.272500,0.261889,0.253369,0.121419,0.045832,0.088719,0.422003,0.2262,...,0.356754,0.306963,0.312494,0.150457,0.066585,0.106456,0.466435,0.253632,0.40042,0.293964
6,<function make_capped_renderer.<locals>.capped...,Qwen/Qwen3-4B-Instruct-2507,0.518333,0.528683,0.489173,0.228994,0.085744,0.164088,0.422003,0.2262,...,0.633691,0.598204,0.591213,0.256264,0.117849,0.184771,0.466435,0.253632,0.40042,0.215648
7,<function make_capped_renderer.<locals>.capped...,Qwen/Qwen3-4B-Instruct-2507,0.497500,0.520639,0.482366,0.229570,0.086051,0.166059,0.422003,0.2262,...,0.603902,0.603381,0.580235,0.247838,0.113615,0.178144,0.466435,0.253632,0.40042,0.222276
8,<function make_capped_renderer.<locals>.capped...,Qwen/Qwen3-4B-Instruct-2507,0.526667,0.531433,0.499764,0.234877,0.087903,0.166558,0.422003,0.2262,...,0.638808,0.600250,0.596124,0.256282,0.117640,0.184783,0.466435,0.253632,0.40042,0.215637


In [ ]:
df_summary_q_capped = rename_capped_techniques(df_summary_qwen_capped, techniques_capped_str)

In [ ]:
challenge_df_q_capped, approach_df_q_capped, outcome_df_q_capped, union_df_q_capped, overall_df_q_capped = make_three_aspect_tables(
    df_summary_q_capped,
    include_metrics=["precision","recall","f1","rouge1_model","rouge2_model","rougeL_model",
                     "rouge1_oracle","rouge2_oracle","rougeL_oracle","oracle_gap_rougeL"],
    multi_model_columns=False,
)

/tmp/ipython-input-2401783402.py:64: FutureWarning: The provided callable <function mean at 0x7fa83316c900> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .aggregate(agg_fn))
/tmp/ipython-input-2401783402.py:64: FutureWarning: The provided callable <function mean at 0x7fa83316c900> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .aggregate(agg_fn))
/tmp/ipython-input-2401783402.py:64: FutureWarning: The provided callable <function mean at 0x7fa83316c900> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .aggregate(agg_fn))
/tmp/ipython-input-2401783402.py:64: FutureWarning: The provided callable <function mean at 0x

In [ ]:
challenge_df_q_capped

technique,least_to_most_capped,least_to_most_one_shot_capped,scoring_based_capped,scoring_based_one_shot_capped,self_ask_capped,self_ask_one_shot_capped,tool_augmented_capped,tool_augmented_one_shot_capped,vanilla_capped
metric,,,,,,,,,
precision,0.475000,0.519167,0.497500,0.526667,0.272500,0.518333,0.485000,0.472500,0.494167
recall,0.517655,0.562234,0.520639,0.531433,0.261889,0.528683,0.517806,0.505849,0.527988
f1,0.467769,0.511057,0.482366,0.499764,0.253369,0.489173,0.475120,0.459696,0.480452
oracle_gap_rougeL,0.209197,0.197181,0.205367,0.204868,0.282707,0.207338,0.207587,0.210839,0.204413
rouge1_model,0.225812,0.235944,0.229570,0.234877,0.121419,0.228994,0.227994,0.227029,0.231795
rouge1_oracle,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003
rouge2_model,0.083070,0.094881,0.086051,0.087903,0.045832,0.085744,0.084669,0.081337,0.086833
rouge2_oracle,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200
rougeL_model,0.162229,0.174245,0.166059,0.166558,0.088719,0.164088,0.163839,0.160586,0.167013


- consistently to expectations, the precision improves and the recall drops. Another notable improvement is given by the stability across models.

- least-to-most-one-shot, scoring-based-one-shot and self-ask-one-shot work still best. Self-ask-one-shot also outperforms vanilla in terms of recall.

- the oracle gap is still quite big, indicating that improvement is possible.



In [ ]:
approach_df_q_capped

technique,least_to_most_capped,least_to_most_one_shot_capped,scoring_based_capped,scoring_based_one_shot_capped,self_ask_capped,self_ask_one_shot_capped,tool_augmented_capped,tool_augmented_one_shot_capped,vanilla_capped
metric,,,,,,,,,
precision,0.603667,0.618667,0.589000,0.606000,0.242500,0.620833,0.600000,0.623000,0.600500
recall,0.620169,0.613189,0.619700,0.599521,0.231768,0.602268,0.616165,0.590946,0.617962
f1,0.587916,0.592422,0.580326,0.581339,0.228547,0.589442,0.583102,0.584022,0.585140
oracle_gap_rougeL,0.276426,0.270168,0.282076,0.272116,0.375868,0.269527,0.274351,0.268206,0.278272
rouge1_model,0.216939,0.223431,0.210328,0.218622,0.088506,0.223267,0.218574,0.227567,0.213747
rouge1_oracle,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775
rouge2_model,0.113787,0.116785,0.112237,0.114385,0.049307,0.116107,0.115038,0.116758,0.111723
rouge2_oracle,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252
rougeL_model,0.169753,0.176010,0.164103,0.174063,0.070311,0.176651,0.171827,0.177973,0.167907


- Best capped F1 ≈ 0.5 (least_to_most_one_shot_capped).
- self ask (without one shot) performs by far the worst, while in its one-shot version the results are good in terms of both precision and recall
- also here the oracle gap is bigger compared to all the other aspects
- the capped version stabilizes predictions

In [ ]:
outcome_df_q_capped

technique,least_to_most_capped,least_to_most_one_shot_capped,scoring_based_capped,scoring_based_one_shot_capped,self_ask_capped,self_ask_one_shot_capped,tool_augmented_capped,tool_augmented_one_shot_capped,vanilla_capped
metric,,,,,,,,,
precision,0.667500,0.665833,0.664167,0.734167,0.447500,0.716667,0.678333,0.716667,0.659167
recall,0.660354,0.647962,0.655993,0.663188,0.427660,0.657763,0.665676,0.656974,0.654188
f1,0.632955,0.625484,0.628837,0.665530,0.416129,0.655453,0.641534,0.657580,0.625423
oracle_gap_rougeL,0.225722,0.222660,0.222803,0.210710,0.293280,0.210698,0.222503,0.211425,0.225279
rouge1_model,0.269599,0.274074,0.271453,0.283150,0.178199,0.284363,0.272593,0.285936,0.271440
rouge1_oracle,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727
rouge2_model,0.125963,0.127161,0.127762,0.136824,0.082802,0.138420,0.127403,0.135353,0.125817
rouge2_oracle,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506
rougeL_model,0.202750,0.205812,0.205668,0.217761,0.135191,0.217773,0.205968,0.217046,0.203192


- Outcome is the strongest across all predictions, therefore proving once again to be the easiest to detect
- Best F1 in capped: 0.67 (scoring_based_one_shot_capped), very close to non-capped outcome best ≈ 0.69 (self_ask_one_shot, least_to_most, etc.)
- precision seems to generally improve with cap

In [ ]:
union_df_q_capped

technique,least_to_most_capped,least_to_most_one_shot_capped,scoring_based_capped,scoring_based_one_shot_capped,self_ask_capped,self_ask_one_shot_capped,tool_augmented_capped,tool_augmented_one_shot_capped,vanilla_capped
metric,,,,,,,,,
precision,0.654665,0.687526,0.664941,0.688399,0.464515,0.678929,0.661974,0.678719,0.661016
recall,0.614529,0.626458,0.617190,0.606860,0.306536,0.604104,0.617298,0.597333,0.622161
f1,0.625333,0.646776,0.629411,0.637865,0.351931,0.630786,0.629635,0.628765,0.632156
oracle_gap_rougeL,0.181973,0.173535,0.178858,0.174854,0.224002,0.175030,0.179038,0.172764,0.181270
rouge1_model,0.278630,0.288776,0.280001,0.288480,0.213705,0.288433,0.280424,0.291944,0.279296
rouge1_oracle,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235
rouge2_model,0.127477,0.133369,0.128412,0.131450,0.088396,0.131125,0.128170,0.130620,0.127484
rouge2_oracle,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569
rougeL_model,0.173630,0.182069,0.176745,0.180750,0.131602,0.180573,0.176566,0.182839,0.174334


- Overall, predictions do improve compared to the non-capped version. Although the improvement is not as great compared to the Llama one, the stability is for sure noticeable.
- the oracle gap is still between 0.17 and 0.22, which exhibits a lower minimum but also a higher maximum compared to the non-capped version. Therefore, the spread is bigger.
- self-ask-capped shows considerably worse prediction results compared to all other tested techniques.

## Final findings and considerations

- We use the Instruct variants of Llama and Qwen because our evaluation relies on instruction-following behavior. Instruct models are fine-tuned to follow prompts faithfully and produce structured outputs, while base models are optimized mainly for text continuation. This ensures that the outputs are aligned with the request of the task itself.

- Using a larger model (Qwen3-4B as opposed to LLama3.2-3B) drastically improves predictions

- Models tend to overpredict, as stated before as well. This means that using "capping" (therefore constraining the model to predict only a set number of indices which corresponds to the median of the gold labels) improves the metric outputs.

- Least-to-most prompting technique, specifically in its one-shot version, seems to be the most promising technique, performing best (or close-best) across all techniques. This is because the small size of the model does not affect it much, and also the least-to-most format mirrors a structured reasoning chain, which is coherent with the "Instruct" kind of modeling. Furthermore, by going through candidate sentences one-by-one, reduces spurious selections and improves precision

- For the Qwen model, techniques like tool-augmented and self-ask (both in their one shot versions) perform very well, while the same cannot be said about Llama. The reason behind this is mainly the fact that Qwen is bigger (4B parameters vs 3B) and is trained on meta-instructions, which are similar to the ones found in these prompting techniques. Furthermore, the use of one shot boosts performance significantly in Qwen (e.g. In the challenge case, self_ask_capped is 0.27, while its one shot version is 0.51).

- Finally, it is noticeable that one-shot prompting produces good grounding effects, improving the stability by feeding an example taken from the validation set to the prompt. This also improves consistency, and models generate outputs that are shorter and more consistent, reducing noise in sentence indices.

- union vs overall distinction: Union represents, intuitively, the aspect agnostic calculation, concatenating the gold standards and the predictions to calculate precision, recall and f1. The overall label instead represent the posterior mean of aspect-level and aspect-agnostic metrics, to have a more grounded idea of model performance.


### Additional experiment

To add value to our project, we decided to implement, as requested by the project work, an additional model, Qwen2.5-3B, which is a lower-parameter version of the Qwen model. It can be loaded in the same way as the one before.

In [41]:
model_name = "Qwen/Qwen2.5-3B-Instruct" # Or "meta-llama/Llama-3.2-3B-Instruct" || Qwen/Qwen3-4B

model, tokenizer = load_model_and_tokenizer(model_name)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Loaded Qwen model and tokenizer (with BF16).


In [42]:
results_qwen_s = []
for tech in techniques:
    print(f"Model Name: {model_name} | Technique: {tech}")
    res = run_evaluation_with_rouge_and_oracle(
        dataset=dataset,
        model_name=model_name,
        model=model,
        tokenizer=tokenizer,
        prompt_technique=tech,
        gold_all=gold_all,
        aspects=ASPECTS,
        abs_by_id=abs_by_id,
        start=0,
        stop=None,
        verbose=True,
        oracle_max_len=None,
        capped = False
    )
    results_qwen_s.append(res)

df_docs_qwen_s, df_summary_qwen_s = results_to_dataframe(results_qwen_s)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Streaming output truncated to the last 5000 lines.
     Predicted: [1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
     Gold:      [1, 2, 3, 7, 8, 9, 10, 13, 14]
     Metrics:   P=0.64 R=1.00 F1=0.78
     ROUGE F1 (model):  R1=0.223  R2=0.050  RL=0.156
     ROUGE F1 (oracle): R1=0.467 R2=0.038 RL=0.243
     Oracle gap (RL):   +0.087

Doc 44 (id=E06-1015) | #sentences=35
--------------------------------------------------------------------------------
  -> Challenge
     Predicted: [9, 14, 23, 26]
     Gold:      [1, 2, 8, 9, 33]
     Metrics:   P=0.25 R=0.20 F1=0.22
     ROUGE F1 (model):  R1=0.173  R2=0.027  RL=0.093
     ROUGE F1 (oracle): R1=0.466 R2=0.338 RL=0.466
     Oracle gap (RL):   +0.372
  -> Approach 
     Predicted: [3, 8, 10, 18, 20, 22, 24, 26, 27, 32]
     Gold:      [3, 19]
     Metrics:   P=0.10 R=0.50 F1=0.17
     ROUGE F1 (model):  R1=0.076  R2=0.021  RL=0.062
     ROUGE F1 (oracle): R1=0.483 R2=0.321 RL=0.448
     Oracle gap (RL):   +0.386
  -> Outcome  
     Predi

In [43]:
challenge_df_qs, approach_df_qs, outcome_df_qs, union_df_qs, overall_df_qs = make_three_aspect_tables(
    df_summary_qwen_s,
    include_metrics=["precision","recall","f1","rouge1_model","rouge2_model","rougeL_model",
                     "rouge1_oracle","rouge2_oracle","rougeL_oracle","oracle_gap_rougeL"],
    multi_model_columns=False,
)

/tmp/ipython-input-2401783402.py:64: FutureWarning: The provided callable <function mean at 0x7a8fbe96c9a0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .aggregate(agg_fn))
/tmp/ipython-input-2401783402.py:64: FutureWarning: The provided callable <function mean at 0x7a8fbe96c9a0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .aggregate(agg_fn))
/tmp/ipython-input-2401783402.py:64: FutureWarning: The provided callable <function mean at 0x7a8fbe96c9a0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .aggregate(agg_fn))
/tmp/ipython-input-2401783402.py:64: FutureWarning: The provided callable <function mean at 0x

In [44]:
challenge_df_qs

technique,least_to_most_one_shot,least_to_most_prompt,scoring_based_one_shot,scoring_based_prompt,self_ask_one_shot,self_ask_prompt,tool_augmented_one_shot,tool_augmented_prompt,vanilla_prompt
metric,,,,,,,,,
precision,0.051076,0.163521,0.066118,0.132340,0.078317,0.079800,0.100916,0.115655,0.167823
recall,0.115992,0.267008,0.147833,0.242702,0.122056,0.136817,0.142381,0.269599,0.275258
f1,0.065922,0.181357,0.081255,0.154837,0.082248,0.088309,0.098617,0.147518,0.188608
oracle_gap_rougeL,0.344798,0.299777,0.321396,0.302780,0.309778,0.336243,0.315190,0.311945,0.295706
rouge1_model,0.035994,0.100445,0.072972,0.095857,0.085786,0.050548,0.075973,0.084007,0.106887
rouge1_oracle,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003
rouge2_model,0.013075,0.034114,0.019293,0.030976,0.024664,0.019651,0.024900,0.029680,0.036929
rouge2_oracle,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200
rougeL_model,0.026628,0.071648,0.050030,0.068646,0.061648,0.035183,0.056236,0.059481,0.075720


In [45]:
approach_df_qs

technique,least_to_most_one_shot,least_to_most_prompt,scoring_based_one_shot,scoring_based_prompt,self_ask_one_shot,self_ask_prompt,tool_augmented_one_shot,tool_augmented_prompt,vanilla_prompt
metric,,,,,,,,,
precision,0.174236,0.115911,0.198792,0.135487,0.279160,0.043390,0.199296,0.089080,0.089280
recall,0.229514,0.231389,0.308811,0.272893,0.342829,0.087381,0.265242,0.213202,0.189067
f1,0.174303,0.144866,0.224167,0.168613,0.270634,0.054830,0.208906,0.118109,0.115052
oracle_gap_rougeL,0.375267,0.407988,0.374543,0.403093,0.345584,0.432747,0.375076,0.419792,0.417984
rouge1_model,0.089617,0.047272,0.094544,0.055683,0.131166,0.016577,0.092138,0.032976,0.036730
rouge1_oracle,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775
rouge2_model,0.043178,0.025283,0.044482,0.027111,0.061020,0.008092,0.043972,0.016387,0.019203
rouge2_oracle,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252
rougeL_model,0.070911,0.038191,0.071635,0.043086,0.100595,0.013431,0.071102,0.026386,0.028194


In [46]:
union_df_qs

technique,least_to_most_one_shot,least_to_most_prompt,scoring_based_one_shot,scoring_based_prompt,self_ask_one_shot,self_ask_prompt,tool_augmented_one_shot,tool_augmented_prompt,vanilla_prompt
metric,,,,,,,,,
precision,0.426998,0.419435,0.486705,0.448702,0.516029,0.348786,0.496389,0.415934,0.473215
recall,0.296752,0.375214,0.392088,0.398909,0.400071,0.212856,0.377456,0.378367,0.381310
f1,0.317053,0.367066,0.404898,0.394892,0.415344,0.233646,0.393285,0.366657,0.391815
oracle_gap_rougeL,0.216158,0.218436,0.198050,0.218688,0.191189,0.248650,0.196268,0.218319,0.203102
rouge1_model,0.236891,0.229852,0.272193,0.229536,0.282599,0.183629,0.268685,0.227629,0.258988
rouge1_oracle,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235
rouge2_model,0.092259,0.088181,0.099618,0.087655,0.105826,0.070879,0.102834,0.088544,0.096683
rouge2_oracle,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569
rougeL_model,0.139445,0.137168,0.157554,0.136915,0.164415,0.106954,0.159335,0.137285,0.152501


In [47]:
results_qwen_s_capped = []
for tech in techniques_capped:
    print(f"Model Name: {model_name} | Technique: {tech}")
    res = run_evaluation_with_rouge_and_oracle(
        dataset=dataset,
        model_name=model_name,
        model=model,
        tokenizer=tokenizer,
        prompt_technique=tech,
        gold_all=gold_all,
        aspects=ASPECTS,
        abs_by_id=abs_by_id,
        start=0,
        stop=None,
        verbose=True,
        oracle_max_len=None,
        capped = True
    )
    results_qwen_s_capped.append(res)

df_docs_qwen_s_capped, df_summary_qwen_s_capped = results_to_dataframe(results_qwen_s_capped)

Streaming output truncated to the last 5000 lines.
     Predicted: [1, 2, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15]
     Gold:      [1, 2, 3, 7, 8, 9, 10, 13, 14]
     Metrics:   P=0.67 R=0.89 F1=0.76
     ROUGE F1 (model):  R1=0.226  R2=0.054  RL=0.173
     ROUGE F1 (oracle): R1=0.467 R2=0.038 RL=0.243
     Oracle gap (RL):   +0.070

Doc 44 (id=E06-1015) | #sentences=35
--------------------------------------------------------------------------------
  -> Challenge
     Predicted: [2, 9, 14, 26]
     Gold:      [1, 2, 8, 9, 33]
     Metrics:   P=0.50 R=0.40 F1=0.44
     ROUGE F1 (model):  R1=0.222  R2=0.097  RL=0.111
     ROUGE F1 (oracle): R1=0.466 R2=0.338 RL=0.466
     Oracle gap (RL):   +0.355
  -> Approach 
     Predicted: [3, 8, 9, 10, 14, 20]
     Gold:      [3, 19]
     Metrics:   P=0.17 R=0.50 F1=0.25
     ROUGE F1 (model):  R1=0.117  R2=0.030  RL=0.088
     ROUGE F1 (oracle): R1=0.483 R2=0.321 RL=0.448
     Oracle gap (RL):   +0.360
  -> Outcome  
     Predicted: [10, 19, 23, 32]
  

In [50]:
df_summary_qwen_s_capped = rename_capped_techniques(df_summary_qwen_s_capped, techniques_capped_str)

In [51]:
challenge_df_q_s_capped, approach_df_q_s_capped, outcome_df_q_s_capped, union_df_q_s_capped, overall_df_q_s_capped = make_three_aspect_tables(
    df_summary_qwen_s_capped,
    include_metrics=["precision","recall","f1","rouge1_model","rouge2_model","rougeL_model",
                     "rouge1_oracle","rouge2_oracle","rougeL_oracle","oracle_gap_rougeL"],
    multi_model_columns=False,
)

/tmp/ipython-input-2401783402.py:64: FutureWarning: The provided callable <function mean at 0x7a8fbe96c9a0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .aggregate(agg_fn))
/tmp/ipython-input-2401783402.py:64: FutureWarning: The provided callable <function mean at 0x7a8fbe96c9a0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .aggregate(agg_fn))
/tmp/ipython-input-2401783402.py:64: FutureWarning: The provided callable <function mean at 0x7a8fbe96c9a0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .aggregate(agg_fn))
/tmp/ipython-input-2401783402.py:64: FutureWarning: The provided callable <function mean at 0x

In [52]:
challenge_df_q_s_capped

technique,least_to_most_capped,least_to_most_one_shot_capped,scoring_based_capped,scoring_based_one_shot_capped,self_ask_capped,self_ask_one_shot_capped,tool_augmented_capped,tool_augmented_one_shot_capped,vanilla_capped
metric,,,,,,,,,
precision,0.320833,0.341667,0.301667,0.253333,0.341667,0.228333,0.326667,0.254167,0.319167
recall,0.326583,0.340004,0.335313,0.278587,0.374758,0.218206,0.350226,0.250206,0.335599
f1,0.306390,0.322916,0.300217,0.245789,0.339104,0.207539,0.320863,0.237186,0.309354
oracle_gap_rougeL,0.236370,0.233633,0.228026,0.244951,0.227443,0.236145,0.231476,0.246557,0.230221
rouge1_model,0.195520,0.200268,0.199603,0.186316,0.202387,0.198931,0.197098,0.181654,0.199944
rouge1_oracle,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003
rouge2_model,0.062688,0.065623,0.066631,0.057701,0.070312,0.062638,0.065056,0.055720,0.067458
rouge2_oracle,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200
rougeL_model,0.135056,0.137793,0.143400,0.126475,0.143982,0.135281,0.139949,0.124869,0.141205


In [53]:
approach_df_q_s_capped

technique,least_to_most_capped,least_to_most_one_shot_capped,scoring_based_capped,scoring_based_one_shot_capped,self_ask_capped,self_ask_one_shot_capped,tool_augmented_capped,tool_augmented_one_shot_capped,vanilla_capped
metric,,,,,,,,,
precision,0.430000,0.415667,0.435000,0.426000,0.353333,0.441167,0.401667,0.412500,0.431667
recall,0.461140,0.413263,0.467021,0.436140,0.381171,0.430723,0.431167,0.428067,0.467108
f1,0.426687,0.393802,0.430944,0.414365,0.350754,0.412454,0.397414,0.400706,0.429435
oracle_gap_rougeL,0.296003,0.284778,0.295865,0.292909,0.310044,0.282614,0.301800,0.293137,0.298305
rouge1_model,0.197605,0.204660,0.196997,0.202113,0.178142,0.211462,0.187957,0.197786,0.193073
rouge1_oracle,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775
rouge2_model,0.093675,0.093059,0.093051,0.094123,0.080886,0.099346,0.087517,0.091495,0.091910
rouge2_oracle,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252
rougeL_model,0.150175,0.161401,0.150314,0.153270,0.136135,0.163564,0.144379,0.153042,0.147874


In [54]:
outcome_df_q_s_capped

technique,least_to_most_capped,least_to_most_one_shot_capped,scoring_based_capped,scoring_based_one_shot_capped,self_ask_capped,self_ask_one_shot_capped,tool_augmented_capped,tool_augmented_one_shot_capped,vanilla_capped
metric,,,,,,,,,
precision,0.419167,0.549167,0.387500,0.365000,0.425833,0.405833,0.427500,0.455833,0.467500
recall,0.399969,0.384612,0.376096,0.280945,0.413850,0.300833,0.402378,0.384556,0.441386
f1,0.383294,0.408468,0.361485,0.299407,0.397095,0.324992,0.389134,0.384077,0.431643
oracle_gap_rougeL,0.254720,0.227044,0.257896,0.259081,0.255696,0.239315,0.250841,0.232431,0.247012
rouge1_model,0.239547,0.273775,0.235861,0.230803,0.239698,0.255480,0.243490,0.273460,0.245082
rouge1_oracle,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727
rouge2_model,0.096345,0.117083,0.096190,0.083099,0.098366,0.102007,0.100336,0.116292,0.106976
rouge2_oracle,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506
rougeL_model,0.173752,0.201427,0.170576,0.169390,0.172775,0.189156,0.177631,0.196041,0.181459


In [55]:
union_df_q_s_capped

technique,least_to_most_capped,least_to_most_one_shot_capped,scoring_based_capped,scoring_based_one_shot_capped,self_ask_capped,self_ask_one_shot_capped,tool_augmented_capped,tool_augmented_one_shot_capped,vanilla_capped
metric,,,,,,,,,
precision,0.567484,0.580836,0.578418,0.545678,0.575327,0.543078,0.568348,0.568306,0.568043
recall,0.499106,0.468957,0.495090,0.446906,0.490662,0.420683,0.477655,0.465715,0.503514
f1,0.522015,0.506704,0.523360,0.481957,0.519138,0.465985,0.508495,0.503161,0.524255
oracle_gap_rougeL,0.184499,0.184852,0.182281,0.187872,0.180557,0.182214,0.182014,0.182125,0.184897
rouge1_model,0.281158,0.289131,0.284276,0.283300,0.287697,0.299205,0.284836,0.290437,0.279049
rouge1_oracle,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235
rouge2_model,0.120159,0.117487,0.123569,0.114783,0.122174,0.121738,0.120120,0.124765,0.120557
rouge2_oracle,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569
rougeL_model,0.171105,0.170751,0.173323,0.167732,0.175047,0.173389,0.173590,0.173479,0.170706


According to expectations, the results here are a middleground between what was observed with Llama (3B parameters) and Qwen3 (4B parameters).

- In terms of computational power, the time needes is closer to the one of Llama, given the fact that the two models share the same amount of parameters.

- Because of the size of the model, there is a noticeable drop in performance in terms of precision, accuracy and F1 compared to Qwen3. This further enforces the idea that bigger models are able to perform this type of tasks easier.

- Consistently to what observed, outcome seems to be the easiest aspect to detect while challenge is the hardest.

- The oracle rouge gap remains consistent across experiments and comparable with the other observed ones.

- The capped variant produces noticeable improvements, even more so that in Qwen3, which is in line with expectations.

- According with expectations, much like in Qwen3 and unline Llama, all prompting techniques produce decent results, with self-ask-one-shot and tool-augmented-one-shot performing fairly well. More so, all the models except the self-ask without the one shot variant perform in a comparable way.